# Linear Programming Seating Arrangement Script 

# Define Parameters

In [949]:
import time 
import math
start_time = time.time()

EventID = 221

# Which machine are you accessing from?
EC2_access = True
parallels_access =    False


# Do you want to run the models or bring them in from the past runs? - True means run them 
run_models = False  

# EC2_access = True
# parallels_access = False


##### Sessions 

In [950]:
# This session is 
lunch_session =  8 
# Manager and allocator only sessions - matching on 
MAonly_session = 4 
dive_deeper_sessions = [] 
# The classic logic 
main_sessions = [1,2,3,5,6,7,8] 

In [951]:
# Which linear progamming API am I using? 
gurobi_lp = True


# Pairs cannot sit together more than this number of times in the MAIN sessions 
upper_count_pair_counts = 1


# Variables for event 
number_sessions =  8 
max_table_size_dict  = {1:9, 2:9, 3:9, 4:9, 5:9, 6:9, 7:9, 8:10}
max_number_tables = 21


# When this is true the scheduling conflicts are included, 
# if changed to false they are removed and everyone is included 
scheduling_conflicts_on = True


# Reporting Outputs 
write_to_DB = False 
write_to_excel = True

cancelled_contacts = []

## Import Packages 

In [952]:
# Numerical and Data Manipulation
import numpy as np 
import pandas as pd 
import math
import statistics
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Functional Programming
from functools import reduce 
import json 
import pickle 

# Copying and Randomization
import copy
import random

# Iteration and Combinations
import itertools
from itertools import repeat, combinations, permutations, combinations_with_replacement

# Collections
from collections import defaultdict, Counter

# Excel and Data Handling
import xlsxwriter
from openpyxl.utils import get_column_letter

# Database Connectivity
import pyodbc
import sqlalchemy as db
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String

# Text Manipulation
import re

# Date and Time
from datetime import datetime, timedelta

# Progress Bar
from tqdm import tqdm

# String Matching
from fuzzywuzzy import fuzz

# Word Document Manipulation
from docx import Document
from docx.shared import Cm

# Graph Theory
import networkx as nx

# Linear Programming
import pulp
from pulp import lpSum, LpMaximize
import gurobipy as grb
from gurobipy import Model, GRB

# System and Process Information
import psutil
import sys

# Excel and Data Handling
import xlsxwriter

# Set random seed for reproducibility
np.random.seed(24)

# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

import string


## Define Functions

In [954]:
# Sorts the (A) and (F) in the dataframe
def sort_group_by_marker(group, session_columns):
    # Extract the last 3 characters from each session column
    group[session_columns] = group[session_columns].apply(lambda x: x.str[-3:])
    
    # Create a list of session columns to use for sorting
    sort_columns = list(session_columns)
    
    # Sort the DataFrame by the specified session columns
    group = group.sort_values(by=sort_columns, ascending=True)
    
    # Drop the temporary columns used for sorting
    group = group.drop(session_columns, axis=1)
    
    return group

# Print red bold text 
def print_in_red_and_large(text):
    formatted_text = f"\033[91m\033[1m{text}\033[0m"
    print(formatted_text)


# Write a table to the database
def create_database_table(table_name, df_name, column_names, engine):
    # Create the metadata object
    metadata = MetaData()

    # Define the table
    table = Table(
        table_name,
        metadata,
        Column('ID', Integer, primary_key=True),  # Add an ID column as primary key
        *(Column(col, String(255), nullable=False) for col in column_names)
    )

    # Create the table in the database
    metadata.create_all(engine)

    # Ensure the table is created
    tables = pd.read_sql("select * from INFORMATION_SCHEMA.TABLES", engine)
    if table_name in tables['TABLE_NAME'].values:
        print(f"Table '{table_name}' created successfully.")
    else:
        print(f"Error: Table '{table_name}' not found in the database.")

# Changes a name from first_last_A to First Last (A)
def convert_name_format(name):
    parts = name.split('_')
    first_name = parts[0].capitalize()
    last_name = parts[1].capitalize()
    group = parts[2].upper()
    formatted_name = f"{first_name} {last_name}  ({group})"
    return formatted_name

# Define a function to extract the session number
def extract_session_number(title):
    match = re.search(r'#(\d+)', title)
    if match:
        session_number = match.group(1)
        return f"Session {session_number}"
    else:
        return None

# Import Data

#####  Database Connection

In [955]:
## OCDB Connection 

logins_username = ""

logins_password = ""

server =  ""

# Choose Database
database= ""

## Create Engine and Connection to the Database
engine = db.create_engine(
    'mssql+pyodbc://'+logins_username+':'+logins_password+'@'+server+'/'+database+'?driver=SQL Server', 
    use_setinputsizes=False) # This argument got rid of the Error

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+logins_username+';PWD='+ logins_password)

#####  Pull in Fund and Allocator Data to match names for Marker

In [956]:
# Choose Database
database = ""

## Create Engine and Connection to the Database
engine = db.create_engine(
    'mssql+pyodbc://'+logins_username+':'+logins_password+'@'+server+'/'+database+'?driver=SQL Server', 
    use_setinputsizes=False) # This argument got rid of the Error

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+logins_username+';PWD='+ logins_password)

#####  **Pull in the event registrations**

##### platform/DB data

In [957]:
## Queries 
df_database = pd.read_sql('select * from ----', cnxn)

df_database['FirstName'] = df_database['FirstName'].str.strip()
df_database['LastName'] = df_database['LastName'].str.strip()
df_database['Email'] = df_database['Email'].str.lower().str.strip()


# Add contact to database to compare
condition = df_database['CompanyType'] == 'Fund'
df_database.loc[condition, 'Contact'] = df_database.loc[condition, 'FirstName'].str.lower() + '_' + df_database.loc[condition, 'LastName'].str.lower() + "_F"

condition = df_database['CompanyType'] == 'Investor'
df_database.loc[condition, 'Contact'] = df_database.loc[condition, 'FirstName'].str.lower() + '_' + df_database.loc[condition, 'LastName'].str.lower() + "_A"

##### excel reg data


In [958]:
if EC2_access == True:
       df_excel_fund = pd.read_excel(r"", sheet_name = 'Public Managers')
       df_excel_allocator = pd.read_excel(r"", sheet_name = 'Public Allocators')
       df_excel_staff = pd.read_excel(r"", sheet_name = 'Staff')
if parallels_access == True:
       df_excel_fund = pd.read_excel(r"", sheet_name = 'Public Managers')
       df_excel_allocator = pd.read_excel(r"", sheet_name = 'Public Allocators')
       df_excel_staff = pd.read_excel(r"", sheet_name = 'Staff')

In [959]:
# # Subset to only needed columns
df_excel_fund = df_excel_fund[['Email', 'First Name', 'Last Name', '*Role',  'AUM Range', 'Asset Class', 'Sub Asset Clases']]

df_excel_allocator = df_excel_allocator[['Email', 'First Name', 'Last Name', '*Role', 
       '*Description of Organization', 'Manager Size Preference 1',
       'Manager Size Preference 2']]

# Concat them together 
df_excel = pd.concat([df_excel_fund, df_excel_allocator], ignore_index=True)

# Email is my primary key - fold to lower 
df_excel['Email'] = df_excel['Email'].str.lower().str.strip()
df_excel['First Name'] = df_excel['First Name'].str.strip()
df_excel['Last Name'] = df_excel['Last Name'].str.strip()


In [960]:
# If there is no role assigned for this guy, make it fund 
if df_excel.loc[(df_excel['First Name'] == 'George') & (df_excel['Last Name'] == 'Michelakis')]['*Role'].isna().values[0] == True:
    df_excel.loc[(df_excel['First Name'] == 'George') & (df_excel['Last Name'] == 'Michelakis'), '*Role'] = 'Manager'

if df_excel.loc[(df_excel['First Name'] == 'Cyrus') & (df_excel['Last Name'] == 'Shirzadi')]['*Role'].isna().values[0] == True:
    df_excel.loc[(df_excel['First Name'] == 'Cyrus') & (df_excel['Last Name'] == 'Shirzadi'), '*Role'] = 'Manager'

In [961]:
# Add contact to database to compare
condition = df_excel['*Role'] == 'Manager'
df_excel.loc[condition, 'Contact'] = df_excel.loc[condition, 'First Name'].str.lower() + '_' + df_excel.loc[condition, 'Last Name'].str.lower() + "_F"

condition = df_excel['*Role'] == 'Allocator'
df_excel.loc[condition, 'Contact'] = df_excel.loc[condition, 'First Name'].str.lower() + '_' + df_excel.loc[condition, 'Last Name'].str.lower() + "_A"

#####  remove cancelled people 

In [962]:
df_excel = df_excel[~df_excel['Contact'].isin(cancelled_contacts)]
df_database = df_database[~df_database['Contact'].isin(cancelled_contacts)]

#####  Create list of facilitators 

In [963]:
# Filter to just the facilitators
df_excel_staff = df_excel_staff[df_excel_staff['Faciliator']=='Y']

# Add contact
df_excel_staff['Contact'] = df_excel_staff['First Name'].str.strip().str.lower() + '_' + df_excel_staff['Last Name'].str.strip().str.lower() + "_S"
facilitators = df_excel_staff['Contact'].unique().tolist()

if len(facilitators) == 19:
    facilitators.append('testing_testing_S')


Merge database data and excel data 

In [964]:
df_db = df_database.merge(df_excel, on = 'Contact', how = 'left', suffixes=['_db', '_excel']) 


# Where one of them is not NA, are they always equal? If there are no rows returned that means yes! 
if df_db[df_db['AUM Range_excel'].notna() & df_db['AUM Range_db'].notna() & (df_db['AUM Range_db'] != df_db['AUM Range_excel'])][['AUM Range_db', 'AUM Range_excel']].shape[0] >0:
    raise ValueError('!!')

# because there are a TON more empty values in excel, im using the DB version
# df_database['AUM Range'].isna().sum(), df_excel['AUM Range'].isna().sum()
df_db =df_db.drop(columns = ['AUM Range_excel', 'Email_excel', 'Email_db']).rename(columns = {'AUM Range_db':'AUM Range'})


#####  update connection - pulling from different database

In [965]:
# Choose Database
database = 'FundsForFood-Prod'

## Create Engine and Connection to the Database
engine = db.create_engine(
    'mssql+pyodbc://'+logins_username+':'+logins_password+'@'+server+'/'+database+'?driver=SQL Server', 
    use_setinputsizes=False) # This argument got rid of the Error

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+logins_username+';PWD='+ logins_password)

##### Scheduling Conflicts

In [966]:
conflicts_query = f'''
Select ads.AgendaDaySessionID, ads.Title, adsi.ContactId, adsi.IsDeleted

from [FundsForFood-Prod]..AgendaDaySession ads

Join [FundsForFood-Prod]..AgendaDaySessionInterest adsi

on ads.AgendaDaySessionID  = adsi.AgendaDaySessionID

Where EventID = {EventID}

and ads.Title like '%Session #%'

and adsi.IsDeleted = 0

Group by ads.AgendaDaySessionID,ads.Title, adsi.ContactId, adsi.IsDeleted

Order by adsi.ContactID asc, ads.Title asc'''


scheduling_conflicts_df_db = pd.read_sql(conflicts_query, cnxn)


for each contactID I want to add Lunch! 
- Lunch availability is not collected since it is not considered a "session" yet it is to us 



In [967]:
# Create a new df with the same columns
df = pd.DataFrame(columns = scheduling_conflicts_df_db.columns)
df['ContactId'] = scheduling_conflicts_df_db['ContactId'].unique()
df['Title'] = 'Session #8: Lunch'
df['IsDeleted'] = False
df['AgendaDaySessionID'] = 0

scheduling_conflicts_df_db = pd.concat([scheduling_conflicts_df_db, df]).sort_values(by = ["ContactId", 'Title'])

In [968]:
## Close Codf_dbnnection
cnxn.close()

# Preprocess

#### Preprocess Fund and Allocator DB Data

In [969]:
# Split into FUnd and allocator df
allocator_df_db  =  df_db[df_db['CompanyType'] == 'Investor']
fund_df_db  =  df_db[df_db['CompanyType'] == 'Fund']

In [970]:
#Drop columns that are completely null
# allocator_data_db = allocator_df_db.dropna(axis='columns', how='all')
# fund_data_db = fund_data_db.dropna(axis='columns', how='all')
allocator_data_db = allocator_df_db
fund_data_db = fund_df_db

# Create a contact field 
fund_data_db['Contact'] = fund_data_db['FirstName'].str.lower() + '_' + fund_data_db['LastName'].str.lower() + '_F'
allocator_data_db['Contact'] = allocator_data_db['FirstName'].str.lower() + '_' + allocator_data_db['LastName'].str.lower() + '_A'

# Fold first and last name to lower 
fund_data_db['FirstName'] = fund_data_db['FirstName'].str.lower()
fund_data_db['LastName'] = fund_data_db['LastName'].str.lower()

fund_data_db['AUM Range'] =fund_data_db['AUM Range'].str.strip()

allocator_data_db['FirstName'] = allocator_data_db['FirstName'].str.lower()
allocator_data_db['LastName'] = allocator_data_db['LastName'].str.lower()

# Rename to match what I have been using in the following code
allocator_data_db = allocator_data_db.rename(columns = {'Asset Class 1': 'Asset Class Theme Pref 1','Asset Class 2': 'Asset Class Theme Pref 2' })

In [971]:
allocator_data_db['Preference'] = allocator_data_db['Preference'].fillna('')
allocator_data_db['NoPreference'] = allocator_data_db['NoPreference'].fillna('')

In [974]:
if df_db[~df_db['Contact'].isin(df_excel['Contact'].unique())].shape[0] >1: # changed this to greater than 1 since elaine chan is in there 
    raise ValueError ('Excel and DB values do not align!')

##### Preprocess preference and no preference names

In [975]:
# Replace whitespace values with None (null)
allocator_data_db.loc[allocator_data_db['Preference'].str.strip() == '', 'Preference'] = None

if allocator_data_db['Contact'].nunique() == allocator_data_db.shape[0]:
    for contact in allocator_data_db['Contact'].unique():
        contact_rows = allocator_data_db['Contact'] == contact
        if allocator_data_db.loc[contact_rows, 'Preference'].iloc[0] is not None:
            # Get the preference values for the current contact
            names_string = allocator_data_db.loc[contact_rows, 'Preference'].iloc[0]
            names_list = names_string.split(',')

            # Strip whitespace, fold to lowercase, and replace spaces with underscores
            cleaned_names_list = [name.strip().lower().replace(' ', '_') + '_F' for name in names_list]

            # Update the "Preference" column with the formatted names
            allocator_data_db.loc[contact_rows, 'Preference'] = ','.join(cleaned_names_list)

In [976]:
# Replace whitespace values with None (null)
allocator_data_db.loc[allocator_data_db['NoPreference'].str.strip() == '', 'NoPreference'] = None

if allocator_data_db['Contact'].nunique() == allocator_data_db.shape[0]:
    for contact in allocator_data_db['Contact'].unique():
        contact_rows = allocator_data_db['Contact'] == contact
        if allocator_data_db.loc[contact_rows, 'NoPreference'].iloc[0] is not None:
            # Get the preference values for the current contact
            names_string = allocator_data_db.loc[contact_rows, 'NoPreference'].iloc[0]
            names_list = names_string.split(',')

            # Strip whitespace, fold to lowercase, and replace spaces with underscores
            cleaned_names_list = [name.strip().lower().replace(' ', '_') + '_F' for name in names_list]

            # Update the "Preference" column with the formatted names
            allocator_data_db.loc[contact_rows, 'NoPreference'] = ','.join(cleaned_names_list)

In [978]:
fund_data_db['Preference'] = fund_data_db['Preference'].fillna('')

# Replace whitespace values with None (null)
fund_data_db.loc[fund_data_db['Preference'].str.strip() == '', 'Preference'] = None

if fund_data_db['Contact'].nunique() == fund_data_db.shape[0]:
    print('ok')
    for contact in fund_data_db['Contact'].unique():
        contact_rows = fund_data_db['Contact'] == contact
        if fund_data_db.loc[contact_rows, 'Preference'].iloc[0] is not None:
            # Get the preference values for the current contact
            names_string = fund_data_db.loc[contact_rows, 'Preference'].iloc[0]
            names_list = names_string.split(',')

            # Strip whitespace, fold to lowercase, and replace spaces with underscores
            cleaned_names_list = [name.strip().lower().replace(' ', '_') + '_A' for name in names_list]

            # Update the "Preference" column with the formatted names
            fund_data_db.loc[contact_rows, 'Preference'] = ','.join(cleaned_names_list)

ok


In [980]:
# # fold to Lowercase and strip whitespace before merge 
# fund_data_db['Email']  = fund_data_db['Email'].str.lower().str.strip()
# allocator_data_db['Email']  = allocator_data_db['Email'].str.lower().str.strip()

In [981]:
allocator_data_db['NoPreference'] = allocator_data_db['NoPreference'].str.replace('__', '_')
allocator_data_db['Preference'] = allocator_data_db['Preference'].str.replace('__', '_')

fund_data_db['NoPreference'] = fund_data_db['NoPreference'].str.replace('__', '_')
fund_data_db['Preference'] = fund_data_db['Preference'].str.replace('__', '_')

##### *Remove Cancelled Contacts from prefrences*

In [982]:
for name in cancelled_contacts:
    if name.endswith('_F'):
        allocator_data_db['Preference'] = allocator_data_db['Preference'].str.replace(name, '')
        allocator_data_db['Preference'] = allocator_data_db['Preference'].str.replace(',,', ',')

        allocator_data_db['NoPreference'] = allocator_data_db['NoPreference'].str.replace(name, '')
        allocator_data_db['NoPreference'] = allocator_data_db['NoPreference'].str.replace(',,', ',')
    elif name.endswith('_A'):
        fund_data_db['Preference'] = fund_data_db['Preference'].str.replace(name, '')
        fund_data_db['Preference'] = fund_data_db['Preference'].str.replace(',,', ',')

#### Scheduling Conflicts DB Data

##### Preprocessing Scheduling conflicts data

In [983]:
# are there any contactID values used for funds and allocators?

set(fund_df_db['ContactID'].unique()).intersection(set(allocator_df_db['ContactID'].unique()))

set()

In [984]:
if scheduling_conflicts_on:
    # list of funds and allocators
    fund_and_allocator_contacts = list(set(fund_df_db['ContactID'].unique()) | set(allocator_df_db['ContactID'].unique()))

    # Filter down to only contacts that are relevant
    scheduling_conflicts_df = scheduling_conflicts_df_db[scheduling_conflicts_df_db['ContactId'].isin(fund_and_allocator_contacts)]

    # Apply the function to create the 'Session' column
    # function is in the define functions area
    scheduling_conflicts_df['Session'] = scheduling_conflicts_df['Title'].apply(extract_session_number)

    # Assign Role column based on where the contactID lives (fund or allocator)
    scheduling_conflicts_df.loc[scheduling_conflicts_df['ContactId'].isin(fund_data_db['ContactID'].unique()), 'Role'] = 'Fund'
    scheduling_conflicts_df.loc[scheduling_conflicts_df['ContactId'].isin(allocator_data_db['ContactID'].unique()), 'Role'] = 'Allocator'

    scheduling_conflicts_df['FirstName'] = np.nan
    scheduling_conflicts_df['LastName'] = np.nan

    for contact_id in scheduling_conflicts_df['ContactId'].unique():

        # if a fund
        if scheduling_conflicts_df[scheduling_conflicts_df['ContactId'] == contact_id]['Role'].unique()[0] == 'Fund':

            # Pull First name from fund df
            firstname = fund_data_db[fund_data_db['ContactID'] == contact_id]['FirstName'].values[0]
            scheduling_conflicts_df.loc[scheduling_conflicts_df['ContactId'] == contact_id, 'FirstName'] = firstname
            # Pull Last name from fund data
            lastname = fund_data_db[fund_data_db['ContactID'] == contact_id]['LastName'].values[0]
            scheduling_conflicts_df.loc[scheduling_conflicts_df['ContactId'] == contact_id, 'LastName'] = lastname

        # if a allocator
        elif scheduling_conflicts_df[scheduling_conflicts_df['ContactId'] == contact_id]['Role'].unique()[0] == 'Allocator':# Pull First name from allocator data
            scheduling_conflicts_df.loc[scheduling_conflicts_df['ContactId'] == contact_id, 'FirstName'] = allocator_data_db[allocator_data_db['ContactID'] == contact_id]['FirstName'].values[0]
            # Pull Last name from fund data
            scheduling_conflicts_df.loc[scheduling_conflicts_df['ContactId'] == contact_id, 'LastName'] = allocator_data_db[allocator_data_db['ContactID'] == contact_id]['LastName'].values[0]

    # # Rearrange the columns 
    scheduling_conflicts_df = scheduling_conflicts_df[['FirstName', 'LastName', 'Role', 'Session','Title', 'ContactId']]

## Define Dictionaries 

In [985]:
allocators = allocator_data_db['Contact'].unique()
funds = fund_data_db['Contact'].unique()
attendees = list(set(funds).union(set(allocators)))

##### **Preferences**

In [986]:
# Remove cancelled people from prefs
for contact in cancelled_contacts:
    fund_data_db['Preference'] = fund_data_db['Preference'].str.replace(contact, '')
    allocator_data_db['Preference'] = allocator_data_db['Preference'].str.replace(contact, '')
    allocator_data_db['NoPreference'] = allocator_data_db['NoPreference'].str.replace(contact, '')

In [987]:
# Create a dictionary with 'Contact' as keys and 'Preference' as values
fund_prefs_dict = {}

for contact, preference in zip(fund_data_db['Contact'], fund_data_db['Preference']):
    if preference is not None:
        fund_prefs_dict[contact] = preference.split(',')
    if preference is None:
        fund_prefs_dict[contact] = None

if set(list(fund_prefs_dict.keys())) != set(funds):
    raise ValueError('Discrepency with funds in pref dict')

In [988]:
fund_prefs_dict = {key: [value for value in values if value != ''] if values is not None else None for key, values in fund_prefs_dict.items()}

In [989]:
# Create a dictionary with 'Contact' as keys and 'Preference' as values
allocator_prefs_dict = {}

for contact, preference in zip(allocator_data_db['Contact'], allocator_data_db['Preference']):
    if preference is not None:
        allocator_prefs_dict[contact] = preference.split(',')
    if preference is None:
        allocator_prefs_dict[contact] = None

if set(list(allocator_prefs_dict.keys())) != set(allocators):
    raise ValueError('Discrepency with allocators in pref dict')

In [990]:
allocator_prefs_dict = {key: [value for value in values if value != '' and value != 'cfa_F'] if values is not None else None for key, values in allocator_prefs_dict.items()}

##### Create lists of preference pairs
##### - Then sort the tuples to ensure consistency

In [991]:
fund_pref_pairs = [(key, value) for key, values in fund_prefs_dict.items() if values is not None for value in values]
# Sort tuples into ((F,A))
temp_list = []
for pair in fund_pref_pairs:
    fund = [person for person in pair if person.endswith('_F')][0]
    allocator = [person for person in pair if person.endswith('_A')][0]
    temp_list.append((fund,allocator))
fund_pref_pairs = temp_list
# Remove tuples where the preferred allocator is not in this event 
fund_pref_pairs = [pair for pair in fund_pref_pairs if any(item in allocators for item in pair)]

In [992]:
allocator_pref_pairs = [(key, value) for key, values in allocator_prefs_dict.items() if values is not None for value in values]
# Sort tuples into ((F,A))
temp_list = []
for pair in allocator_pref_pairs:
    fund = [person for person in pair if person.endswith('_F')][0]
    allocator = [person for person in pair if person.endswith('_A')][0]
    temp_list.append((fund,allocator))
allocator_pref_pairs = temp_list
# Remove tuples where the preferred fund is not in this event 
allocator_pref_pairs = [pair for pair in allocator_pref_pairs if any(item in funds for item in pair)]

##### List of mutual preferences

In [993]:
mutual_pref_pairs = set(allocator_pref_pairs).intersection(set(fund_pref_pairs))

##### **Conflicts**


In [994]:
if scheduling_conflicts_on:
    # Separate them based on F or A 
    scheduling_conflicts_A = scheduling_conflicts_df.loc[scheduling_conflicts_df['Role'] == 'Allocator']
    scheduling_conflicts_F = scheduling_conflicts_df.loc[scheduling_conflicts_df['Role'].isin(['Manager', 'Fund'])]

    scheduling_conflicts_A['Contact'] = scheduling_conflicts_A['FirstName'].str.lower().str.strip() + '_' + scheduling_conflicts_A['LastName'].str.lower().str.strip() + '_A'
    scheduling_conflicts_F['Contact'] = scheduling_conflicts_F['FirstName'].str.lower().str.strip() + '_' + scheduling_conflicts_F['LastName'].str.lower().str.strip() + '_F'

##### These dictionaries are a list of people that ARE Available in each session

In [996]:
if scheduling_conflicts_on:
    # Allocator
    allocator_scheduling_conflicts = {}
    for i in range(1, number_sessions+1):
        session_name = f"Session {i}"
        allocator_scheduling_conflicts [f"meeting_{i}_conflicts"] = list(set((scheduling_conflicts_A.loc[scheduling_conflicts_A['Session'] == session_name])['Contact'].to_list()))

    # if the contactID is NOT in scheduling_conflicts_A, add the name to every session 
    allocators_without_availability  = allocator_data_db[allocator_data_db['ContactID'].isin(list(set(allocator_data_db['ContactID'].unique())- set(scheduling_conflicts_A['ContactId'].unique())))]['Contact'].tolist()
    allocators_without_availability.remove('ian_smith_A')
    for i in range(1, number_sessions+1):
        allocator_scheduling_conflicts [f"meeting_{i}_conflicts"] = allocator_scheduling_conflicts [f"meeting_{i}_conflicts"] + allocators_without_availability


    # Funds 
    fund_scheduling_conflicts = {}
    for i in range(1, number_sessions+1):
        session_name = f"Session {i}"
        fund_scheduling_conflicts [f"meeting_{i}_conflicts"] = list(set((scheduling_conflicts_F.loc[scheduling_conflicts_F['Session'] == session_name])['Contact'].to_list()))

    # if the contactID is NOT in scheduling_conflicts_F, add the name to every session 
    funds_without_availability  = fund_data_db[fund_data_db['ContactID'].isin(list(set(fund_data_db['ContactID'].unique())- set(scheduling_conflicts_F['ContactId'].unique())))]['Contact'].tolist()
    for i in range(1, number_sessions+1):
        fund_scheduling_conflicts [f"meeting_{i}_conflicts"] = fund_scheduling_conflicts [f"meeting_{i}_conflicts"] + funds_without_availability


else: 
    allocator_scheduling_conflicts = {}
    for i in range(1, number_sessions+1):
        allocator_scheduling_conflicts [f"meeting_{i}_conflicts"] = set(allocators)

    fund_scheduling_conflicts = {}
    for i in range(1, number_sessions+1):
        fund_scheduling_conflicts [f"meeting_{i}_conflicts"] = set(funds)


In [997]:
for i in range(1, number_sessions+1):
    print('ian_smith_A' in allocator_scheduling_conflicts[f'meeting_{i}_conflicts'])

True
True
True
True
True
True
False
True


In [998]:
# Check for people missing from different sessions
for i in range(1, number_sessions+1):
    print(len(allocator_scheduling_conflicts[f"meeting_{i}_conflicts"]))

85
84
84
83
83
84
77
85


In [999]:
# Check for people missing from different sessions
for i in range(1, number_sessions+1):
    print(len(fund_scheduling_conflicts[f"meeting_{i}_conflicts"]))

40
39
40
40
40
39
39
40


##### **Dictionary of which sessions people ARE available**

In [1000]:
if scheduling_conflicts_on:

    # Allocators 
    allocator_available_sessions ={}
    for contact in scheduling_conflicts_A['Contact'].unique().tolist():
        allocator_available_sessions[contact] = [int(re.search(r'\d+', session).group()) for session in scheduling_conflicts_A[scheduling_conflicts_A['Contact'] == contact]['Session']]

    # For allocators that did not fill out availability - assume available for all 
    for contact in allocators_without_availability:
        allocator_available_sessions[contact] = [i for i in range(1, number_sessions+1)]


    # Funds 
    fund_available_sessions ={}
    for contact in scheduling_conflicts_F['Contact'].unique().tolist():
        fund_available_sessions[contact] = [int(re.search(r'\d+', session).group()) for session in scheduling_conflicts_F[scheduling_conflicts_F['Contact'] == contact]['Session']]

    # For funds that did not fill out availability - assume available for all 
    for contact in funds_without_availability:
        fund_available_sessions[contact] = [i for i in range(1, number_sessions+1)]

else:
    # Create a dictionary called allocator_available_sessions
    allocator_available_sessions = {allocator: list(range(1, number_sessions+1)) for allocator in allocators}
    fund_available_sessions = {fund: list(range(1, number_sessions+1)) for fund in funds}

##### **Allocator No Prefs**

In [1001]:
# Create a dictionary with 'Contact' as keys and 'Preference' as values
allocator_no_prefs_dict = {}

for contact, preference in zip(allocator_data_db['Contact'], allocator_data_db['NoPreference']):
    if preference is not None:
        allocator_no_prefs_dict[contact] = preference.split(',')
    if preference is None:
        allocator_no_prefs_dict[contact] = None

if set(list(allocator_no_prefs_dict.keys())) != set(allocators):
    raise ValueError('Discrepency with allocators in pref dict')

In [1002]:
allocator_no_pref_pairs = [(key, value) for key, values in allocator_no_prefs_dict.items() if values is not None for value in values]
# Sort tuples 
temp_list = []
for pair in allocator_no_pref_pairs:
    fund = [person for person in pair if person.endswith('_F')][0]
    allocator = [person for person in pair if person.endswith('_A')][0]
    temp_list.append((fund,allocator))
allocator_no_pref_pairs = temp_list
# Remove tuples where the preferred fund is not in this event 
allocator_no_pref_pairs = [pair for pair in allocator_no_pref_pairs if any(item in funds for item in pair)]

##### **Pair Counts**
- generate a list of pairs and then sort the tuples for consistent
- Blow out to sessions or tables, look at combinations of tables in sessions and then go from there 
- How many combinations of F/A table structures are there - look at all of these in combination form and then aim to differ the following tables from this. 

In [1003]:
# Generate combinations
pair_count_list = list(combinations(attendees, 2))
pair_count_list = list(set(pair_count_list)) # remove duplicates


# Sort each tuple within the list
temp_list = []
for pair in pair_count_list:
    # If its two funds or two allocators, no need to do anything just add them back in 
    if (pair[0] in funds and pair[1] in funds) or (pair[0] in allocators and pair[1] in allocators):
        temp_list.append((pair[0],pair[1]))
    # If its a fund/allocator pairschedu
    else:
        fund = [person for person in pair if person.endswith('_F')][0]
        allocator = [person for person in pair if person.endswith('_A')][0]
        temp_list.append((fund,allocator))

pair_count_list = temp_list

##### **Fund Allocator Pairs**

In [1004]:
FA_pairs = []
for fund in funds:
    for allocator in allocators:
        FA_pairs.append((fund,allocator))

##### **AUM and Primary Strat pairs**

In [1005]:
aum_matches =  []
aum_no_matches =  []

for pair in FA_pairs:

    fund_contact = [value for value in pair if value.endswith('_F')][0]
    fund_aum = fund_data_db[fund_data_db['Contact'] == fund_contact]['AUM Range'].values[0]

    allocator_contact = [value for value in pair if value.endswith('_A')][0]
    allocator_aum_pref_1 = allocator_data_db[allocator_data_db['Contact'] == allocator_contact]['Manager Size Preference 1'].values[0]
    allocator_aum_pref_2 = allocator_data_db[allocator_data_db['Contact'] == allocator_contact]['Manager Size Preference 2'].values[0]


    # AUM MATCHES 

    # If fund AUM matches Allocator first pick AUM 
    if fund_aum == allocator_aum_pref_1: 
        aum_matches.append([fund_contact, allocator_contact, fund_aum, allocator_aum_pref_1])
        # print(fund_aum, allocator_aum_pref_1)

    # If fund AUM matches Allocator second pick AUM 
    elif fund_aum == allocator_aum_pref_2:
        aum_matches.append([fund_contact, allocator_contact, fund_aum, allocator_aum_pref_2])
        # print(fund_aum, allocator_aum_pref_2)

    # else: 
    #     print(fund_aum,'-', allocator_aum_pref_1,'-', allocator_aum_pref_2)
    # AUM NO MATCHES 
    elif fund_aum  == '$0.5B - $2B' and (allocator_aum_pref_1 in [ '$5B - $10B','Over $10B']  or allocator_aum_pref_2 in [ '$5B - $10B','Over $10B']) :
        aum_no_matches.append([fund_contact, allocator_contact, fund_aum, allocator_aum_pref_1, allocator_aum_pref_2])
        # print(fund_aum,'-', allocator_aum_pref_1,'-', allocator_aum_pref_2)

    elif fund_aum  =='Over $10B' and (allocator_aum_pref_1 in ['$0.5B - $2B', '$2B - $5B']  or allocator_aum_pref_2 in ['$0.5B - $2B', '$2B - $5B']):
        aum_no_matches.append([fund_contact, allocator_contact, fund_aum, allocator_aum_pref_1, allocator_aum_pref_2])
        # print(fund_aum,'-', allocator_aum_pref_1,'-', allocator_aum_pref_2)

# The remaining (below) not matching or not matching, they are in ~limbo~ where they do not need to be minimized or maximized
        
    # else:
    #     if fund_aum is not None: #and not np.isnan(allocator_aum_pref_1) and not np.isnan(allocator_aum_pref_2):
            # print(fund_aum,'-', allocator_aum_pref_1,'-', allocator_aum_pref_2)

# print('if nothing prints, then all pairs where fund and allocator both have a value are either in matches or no matches')

aum_matches_pairs = [(item[0], item[1]) for item in aum_matches]

aum_no_matches_pairs = [(item[0], item[1]) for item in aum_no_matches]

In [1006]:
# this is the one not sorted by pair count list

fund_aum_matches_og= []

for fund1 in funds:
    fund1_aum_class = fund_data_db[fund_data_db['Contact']==fund1]['AUM Range'].values[0]

    for fund2 in funds:
        fund2_aum_class = fund_data_db[fund_data_db['Contact']==fund2]['AUM Range'].values[0]
        if fund1_aum_class == fund2_aum_class and fund1_aum_class != None:
            # print(fund1,fund1_aum_class, fund2, fund2_aum_class )
            fund_aum_matches_og.append((fund1, fund2))

# Sort them by pair count list 
fund_aum_matches = []
for pair in fund_aum_matches_og:
    if pair in pair_count_list:
        fund_aum_matches.append(pair)
        # print('ok')
    elif (pair[1], pair[0]) in pair_count_list:
        fund_aum_matches.append((pair[1], pair[0]))
        # print('flipping!')
    elif pair[1] != pair[0]:
        raise ValueError('uh oh')

In [1007]:
fund_aum_NO_matches_og= []

for fund1 in funds:
    fund1_aum_class = fund_data_db[fund_data_db['Contact']==fund1]['AUM Range'].values[0]

    for fund2 in funds:
        fund2_aum_class = fund_data_db[fund_data_db['Contact']==fund2]['AUM Range'].values[0]

        if (fund1_aum_class == 'Under $0.5B' and fund2_aum_class in  ['$2B - $5B', '$5B - $10B', 'Over $10B']) or (fund2_aum_class == 'Under $0.5B' and fund1_aum_class in  [ '$2B - $5B', '$5B - $10B', 'Over $10B']):
            fund_aum_NO_matches_og.append((fund1, fund2))

        elif (fund1_aum_class == '$0.5B - $2B' and fund2_aum_class in  [ '$5B - $10B', 'Over $10B']) or (fund2_aum_class == '$0.5B - $2B' and fund1_aum_class in  ['$5B - $10B', 'Over $10B']):
            fund_aum_NO_matches_og.append((fund1, fund2))

        elif (fund1_aum_class == '$5B - $10B' and fund2_aum_class in  ['Under $0.5B', '$0.5B - $2B']) or (fund2_aum_class == '$5B - $10B' and fund1_aum_class in  ['Under $0.5B', '$0.5B - $2B']):
            fund_aum_NO_matches_og.append((fund1, fund2))

        elif( fund1_aum_class == 'Over $10B' and fund2_aum_class in  ['Under $0.5B', '$0.5B - $2B', '$2B - $5B']) or ( fund2_aum_class == 'Over $10B' and fund1_aum_class in  ['Under $0.5B', '$0.5B - $2B', '$2B - $5B']):
            fund_aum_NO_matches_og.append((fund1, fund2))

        elif (fund1_aum_class == '$2B - $5B' and fund2_aum_class in  ['Under $0.5B', 'Over $10B']) or (fund2_aum_class == '$2B - $5B' and fund1_aum_class in  ['Under $0.5B', 'Over $10B']):
            fund_aum_NO_matches_og.append((fund1, fund2))

# Sort them by pair count lust 
fund_aum_NO_matches = []
for pair in fund_aum_NO_matches_og:
    if pair in pair_count_list:
        fund_aum_NO_matches.append(pair)
        # print('ok')
    elif (pair[1], pair[0]) in pair_count_list:
        fund_aum_NO_matches.append((pair[1], pair[0]))
        # print('flipping!')
    elif pair[1] != pair[0]:
        raise ValueError('uh oh')

##### **Asset Class Matches**

##### fund to fund asset class matches

In [1008]:
fund_assetclass_matches_og= []

for fund1 in funds:
        fund1_asset_class = fund_data_db[fund_data_db ['Contact']==fund1]['Firm Asset Class'].unique()
        for fund2 in funds:
            fund2_asset_class = fund_data_db[fund_data_db ['Contact']==fund2]['Firm Asset Class'].unique()
            if  fund1_asset_class != None and fund2_asset_class != None and fund1_asset_class == fund2_asset_class: 
                fund_assetclass_matches_og.append((fund1, fund2))

# Sort them by pair count lust 
fund_assetclass_matches = []
for pair in fund_assetclass_matches_og:
    if pair in pair_count_list:
        fund_assetclass_matches.append(pair)
        # print('ok')
    elif (pair[1], pair[0]) in pair_count_list:
        fund_assetclass_matches.append((pair[1], pair[0]))
        # print('flipping!')
    elif pair[1] != pair[0]:
        print('uh oh')

##### **Allocator Institution Matches**

In [1009]:
# if the value is an empty string make it null!
# allocator_data_db.loc[allocator_data_db['Description Of Organization'] == '', 'Description Of Organization'] = None
allocator_data_db.loc[allocator_data_db['*Description of Organization'] == '', '*Description of Organization'] = None

In [1010]:
institution_matches_og =[]

for allocator1 in allocators:
    allocator1_institution_type = allocator_data_db[allocator_data_db['Contact']==allocator1]['*Description of Organization'].values[0]
    for allocator2 in allocators:
        allocator2_institution_type = allocator_data_db[allocator_data_db['Contact']==allocator2]['*Description of Organization'].values[0]
        
        if allocator1_institution_type == allocator2_institution_type:
            # print(allocator1_institution_type, allocator2_institution_type)
            institution_matches_og.append((allocator1, allocator2))

# remove duplicates
institution_matches_og = list(set(institution_matches_og))

# Sort them by pair count lust 
institution_matches = []
for pair in institution_matches_og:
    if pair in pair_count_list:
        institution_matches.append(pair)
        # print('ok')
    elif (pair[1], pair[0]) in pair_count_list:
        institution_matches.append((pair[1], pair[0]))
    elif pair[1] != pair[0]:
        print('uh oh')


## Skeleton of Seating Arrangement 

### Format & Output Dataframes

In [1012]:
number_tables_dict = {}

In [1013]:
table_frameworks = {}
for session in range(1, number_sessions+1):
    if session != 8:
        table_frameworks[f'Session {session}'] = {}
        for table in range(1, number_tables_dict[session]+1):
            # print(table)
            table_frameworks[f'Session {session}'][f'Table {table}'] = ()
            assignments = adjusted_frameworks_df[(adjusted_frameworks_df['Table'] == f'Table {table}') & (adjusted_frameworks_df['Session'].str.contains(f'{session}'))]['Company Type'].dropna().tolist()
            assignments = [value for value in assignments if value != 'Facilitator']
            assignments = sorted(['F' if value == 'Manager' else 'A' for value in assignments])
            table_frameworks[f'Session {session}'][f'Table {table}'] = tuple(assignments)

table_frameworks[f'Session 8'] = {}
for table in range(1, number_tables_dict[8]+1):
    assignments = adjusted_frameworks_df[(adjusted_frameworks_df['Table'] == f'Table {table}') & (adjusted_frameworks_df['Session'].str.contains(f'Lunch'))]['Company Type'].dropna().tolist()
    assignments = [value for value in assignments if value != 'Facilitator']
    assignments = sorted(['F' if value == 'Manager' else 'A' for value in assignments])
    table_frameworks[f'Session {8}'][f'Table {table}'] = tuple(assignments)

In [1014]:
for i in range(1, number_sessions + 1):
    session_tables = table_frameworks[f'Session {i}']
    print(f'Session {i} : All tables are seated?',
        len(list(session_tables.values())) == number_tables_dict[i])

Session 1 : All tables are seated? True
Session 2 : All tables are seated? True
Session 3 : All tables are seated? True
Session 4 : All tables are seated? True
Session 5 : All tables are seated? True
Session 6 : All tables are seated? True
Session 7 : All tables are seated? True
Session 8 : All tables are seated? True


In [1015]:
table_size_dict = {}

for i in range(1, number_sessions+1):
    # create a subdictionary for this session 
    table_size_dict[i] = {}
    framework = table_frameworks[f'Session {i}'] 
    for table in framework:
        # table_size_dict[i][table] = {}
        assignments = framework[table]
        table_size_dict[i][table] = len(assignments)
        # table_size_dict[i][table]['A'] = assignments.count('A')
        # table_size_dict[i][table]['F'] = assignments.count('F')

##### Quality Check - Check that all tables are utilized in each session

In [1016]:
for i in range(1, number_sessions + 1):
    session_tables = table_frameworks[f'Session {i}']
    print(f'Session {i} : All tables are seated?',
        len(list(session_tables.values())) == number_tables_dict[i])

Session 1 : All tables are seated? True
Session 2 : All tables are seated? True
Session 3 : All tables are seated? True
Session 4 : All tables are seated? True
Session 5 : All tables are seated? True
Session 6 : All tables are seated? True
Session 7 : All tables are seated? True
Session 8 : All tables are seated? True


### Format & Output Dataframes

In [1017]:
## Create dataframe from dictionary
table_frameworks_df = pd.DataFrame(table_frameworks)

### Total Ratio Value Counts 

In [1018]:
# Create a dictionary to store the counts for each ratio
ratio_counts = {}

# Iterate over sessions and tables

for session, tables in table_frameworks.items():
    for table, contents in tables.items():
        # Calculate the ratio of A to F for each table
        ratio = f"{contents.count('A')}/{contents.count('F')}"
        
        # Increment the count in the dictionary
        ratio_counts.setdefault(ratio, {}).setdefault(session, 0)
        ratio_counts[ratio][session] += 1

# Convert the dictionary to a DataFrame
df_counts = pd.DataFrame(ratio_counts)

# Transpose the DataFrame for the desired format
ratio_counts_df = df_counts.T.fillna(0).astype(int).sort_index()

# Rearrange to be chronological
ratio_counts_df = ratio_counts_df[[f'Session {i}' for i in range(1, number_sessions+1)]]

# print
ratio_counts_df

,Session 1,Session 2,Session 3,Session 4,Session 5,Session 6,Session 7,Session 8
0/6,0,0,0,2,0,0,0,0
0/7,0,0,0,4,0,0,0,0
3/2,0,0,0,0,0,0,2,0
4/2,15,12,16,0,17,12,15,0
5/2,5,6,4,0,3,6,1,8
5/3,0,0,0,0,0,1,1,3
6/0,0,0,0,3,0,0,0,0
6/3,0,1,0,0,0,0,0,5
7/0,0,0,0,7,0,0,0,0
8/0,0,0,0,2,0,0,0,0


Add the number of Tables for each Session 

In [1019]:
number_tables_df = pd.DataFrame(number_tables_dict.items(), columns=['Session', 'Number of Tables']).set_index('Session').T
number_tables_df.columns = ratio_counts_df.columns

# Add table counts to ratio 
ratio_counts_df = pd.concat([number_tables_df, ratio_counts_df])

# Set secondary column index as Number of Tables 
ratio_counts_df = ratio_counts_df.T.reset_index().rename(columns = {'index':'Session'}).set_index(['Session', 'Number of Tables']).T

### Create DataFrame for Total counts per Session

In [1020]:
# Initialize a dictionary to store the counts
counts_dict = {}

# Iterate through sessions and tables
for session, tables in table_frameworks.items():
    session_counts = {'Funds': 0, 'Allocators': 0}
    
    for table, contents in tables.items():
        # Count the occurrences of 'A' and 'F' in each table
        session_counts['Funds'] += contents.count('F')
        session_counts['Allocators'] += contents.count('A')

    # Store the counts in the dictionary
    counts_dict[(session, 'Funds')] = session_counts['Funds']
    counts_dict[(session, 'Allocators')] = session_counts['Allocators']
    
data = counts_dict

In [1021]:
# Initialize a dictionary to store the counts
data = {}

# Iterate through sessions and tables
for session, tables in table_frameworks.items():
    session_counts = {'Funds': 0, 'Allocators': 0}
    
    for table, contents in tables.items():
        # Count the occurrences of 'A' and 'F' in each table
        session_counts['Funds'] += contents.count('F')
        session_counts['Allocators'] += contents.count('A')

    # Store the counts in the dictionary
    data[(session, 'Funds')] = session_counts['Funds']
    data[(session, 'Allocators')] = session_counts['Allocators']


# Create a dataframe from the dictionary
FA_counts_df = pd.DataFrame.from_dict(data, orient='index', columns=['Value'])

# Rename the row and column indices
FA_counts_df.index.names = ['Session']

# Print the dataframe
FA_counts_df = FA_counts_df.reset_index()

FA_counts_df['Session'] = FA_counts_df['Session'].apply(str)
FA_counts_df[['Session', 'Role']] = FA_counts_df['Session'].str.split(',', expand=True)
FA_counts_df['Session'] = FA_counts_df['Session'].str[2:-1].str.strip()
FA_counts_df['Role'] = FA_counts_df['Role'].str[2:-2].str.strip()
FA_counts_df = FA_counts_df.rename(columns = {'Value':'Count'})

# Set index as Session and Role
FA_counts_df = FA_counts_df.set_index(['Session', 'Role'])

# pivot the table so that Sessions are across the top and Funds/Allocators are rows
FA_counts_df = FA_counts_df.reset_index().pivot(index='Role', columns='Session', values='Count')

# filter to only rows that have 'Fund' or 'Allocator'
FA_counts_df = FA_counts_df[FA_counts_df.index.isin(['Funds', 'Allocators'])]

# remove the name of the columns index
FA_counts_df.columns.name = None

# Preview df
FA_counts_df

,Session 1,Session 2,Session 3,Session 4,Session 5,Session 6,Session 7,Session 8
Role,,,,,,,,
Allocators,85,84,84,83,83,83,76,85
Funds,40,39,40,40,40,39,39,40


## Bring in the Seating Assignments that Hank Sent over 

In [1022]:
adjusted_frameworks_df['Contact'] = adjusted_frameworks_df['Name'].str.lower().str.strip().str.replace("  ", ' ').str.replace(' ', '_')

adjusted_frameworks_df.loc[adjusted_frameworks_df['Company Type'] == 'Manager', 'Contact'] = adjusted_frameworks_df['Contact'] + '_F'
adjusted_frameworks_df.loc[adjusted_frameworks_df['Company Type'] == 'Allocator', 'Contact'] = adjusted_frameworks_df['Contact'] + '_A'
adjusted_frameworks_df.loc[adjusted_frameworks_df['Company Type'] == 'Facilitator', 'Contact'] = adjusted_frameworks_df['Contact'] + '_S'

In [1023]:
meeting_assignments  = {}
for session in range(1, number_sessions+1):
    if session != 8:
        meeting_assignments[f'Session {session}'] = {}
        for table in range(1, number_tables_dict[session]+1):
            # print(table)
            # meeting_assignments[f'Session {session}'][table]= []
            assignments = adjusted_frameworks_df[(adjusted_frameworks_df['Table'] == f'Table {table}') & (adjusted_frameworks_df['Session'].str.contains(f'{session}'))]['Contact'].dropna().tolist()
            meeting_assignments[f'Session {session}'][table]= assignments


In [1024]:
meeting_assignments[f'Session 8'] = {}
for table in range(1, number_tables_dict[8]+1):
    for seat in range(1, 11): 

        assignments = adjusted_frameworks_df[(adjusted_frameworks_df['Table'] == f'Table {table}') & (adjusted_frameworks_df['Session'].str.contains(f'Lunch'))]['Contact'].dropna().tolist()
        meeting_assignments[f'Session 8'][table]= assignments

SyntaxError: expected ':' (2173944595.py, line 3)

In [1032]:
seat = 5

table = 6
session = 6

In [1054]:
meeting_assignments['Session 8'] = {}

for table in range(1, number_tables_dict[8]+1):
    meeting_assignments['Session 8'][table] = {}
    for seat in range(1, 11):
        name = adjusted_frameworks_df[(adjusted_frameworks_df['Table'] == f'Table {table}') & (adjusted_frameworks_df['Session'].str.contains('Lunch'))& (adjusted_frameworks_df['Seat'] == f'{seat}')]['Contact'].values[0]
        if pd.notna(name):
            meeting_assignments['Session 8'][table][seat] = name
        else:
            meeting_assignments['Session 8'][table][seat] = ()

# assignments = [value for value in assignments if not pd.isna(value)]
# print(assignments)

## Lunch

In [1065]:
lunch_table_assignments = {}
lunch_table_assignments['Session 8'] = meeting_assignments['Session 8']

In [1068]:
# Create dictionary to store pair counts # 
lunch_pair_counts = {}

# Iterate through sessions and table assignments
for session, table_assignments in lunch_table_assignments.items():
    # Iterate through tables and assigned names
    for table, assigned_names in table_assignments.items():
        # print(assigned_names)
        # Iterate through pairs of assigned names
        for seat_pairs in [(i, i % 10 + 1) for i in range(1, 11)]:
            if assigned_names[seat_pairs[0]] != () and assigned_names[seat_pairs[1]] != ():
                lunch_pair_counts[(assigned_names[seat_pairs[0]], assigned_names[seat_pairs[1]])] = 1

In [1069]:
lunch_satisfied_allocator_pref_pairs = list(set([pair for pair in allocator_pref_pairs if pair in lunch_pair_counts or (pair[1], pair[0]) in lunch_pair_counts]))
lunch_satisfied_fund_pref_pairs =list(set([pair for pair in fund_pref_pairs if pair in lunch_pair_counts or (pair[1], pair[0]) in lunch_pair_counts]))
lunch_satisfied_mutual_pref_pairs = list(set([pair for pair in mutual_pref_pairs if pair in lunch_pair_counts  or (pair[1], pair[0]) in lunch_pair_counts]))
lunch_satisfied_fund_assetclass_pairs = list(set([pair for pair in fund_assetclass_matches if pair in lunch_pair_counts  or (pair[1], pair[0]) in lunch_pair_counts]))
lunch_satisfied_fund_aum_pairs = list(set([pair for pair in fund_aum_matches if pair in lunch_pair_counts  or (pair[1], pair[0]) in lunch_pair_counts]))
lunch_satisfied_allocator_nopref_pairs = list(set([pair for pair in allocator_no_pref_pairs if pair in lunch_pair_counts  or (pair[1], pair[0]) in lunch_pair_counts]))

In [680]:
# remaining_mutual_pref_pairs = [pair for pair in mutual_pref_pairs if pair not in lunch_satisfied_mutual_pref_pairs ]
# remaining_allocator_pref_pairs =[pair for pair in allocator_pref_pairs if pair not in lunch_satisfied_allocator_pref_pairs]
# remaining_fund_pref_pairs = [pair for pair in fund_pref_pairs if pair not in lunch_satisfied_fund_pref_pairs ]

In [1070]:
print('LUNCH')
if  len(allocator_pref_pairs) > 0:
    print(round((len(set(lunch_satisfied_allocator_pref_pairs)) / len(allocator_pref_pairs) )*100, 2) , '% Allocator prefs satsified  in Lunch')
if  len(fund_pref_pairs) > 0:
    print(round((len(set(lunch_satisfied_fund_pref_pairs)) / len(fund_pref_pairs))*100, 2) , '% Fund prefs satsified in Lunch')
if  len(mutual_pref_pairs) > 0:
    print(round((len(set(lunch_satisfied_mutual_pref_pairs)) / len(mutual_pref_pairs))*100, 2) , '% Mutual prefs satsified in Lunch')
if  len(allocator_no_pref_pairs) > 0:
    print(round((len(set(lunch_satisfied_allocator_nopref_pairs)) / len(allocator_no_pref_pairs))*100, 2), '% No prefs satsified  in Lunch')


LUNCH
13.59 % Allocator prefs satsified  in Lunch
4.9 % Fund prefs satsified in Lunch
8.33 % Mutual prefs satsified in Lunch
0.0 % No prefs satsified  in Lunch


In [1071]:
# Create dictionary to store pair counts # 
lunch_table_pair_counts = {}
for pair in pair_count_list:
    for table in lunch_table_assignments['Session 8']:
        assignments = lunch_table_assignments['Session 8'][table]
        if pair[0] in assignments and pair[1] in assignments:
            lunch_table_pair_counts[pair] = 1

# Manager / Allocator only Session
##### - match on institution type for A 


Added consideration: 
- For institution types that have multiple tables, try to match on AUM as well
- E&F has two tables
- OCIO has two tables

In [1072]:
EandF_allocators = allocator_data_db[allocator_data_db['*Description of Organization'] == 'E&F']['Contact'].to_list()
Pension_allocators = allocator_data_db[allocator_data_db['*Description of Organization'] == 'Pension']['Contact'].to_list()
OCIO_allocators = allocator_data_db[allocator_data_db['*Description of Organization'] == 'OCIO']['Contact'].to_list()
SFO_allocators = allocator_data_db[allocator_data_db['*Description of Organization'] == 'SFO']['Contact'].to_list()
MFO_allocators = allocator_data_db[allocator_data_db['*Description of Organization'] == 'MFO']['Contact'].to_list()
Hospital_allocators = allocator_data_db[allocator_data_db['*Description of Organization'] == 'Hospital']['Contact'].to_list()
Consultant_allocators = allocator_data_db[allocator_data_db['*Description of Organization'] == 'Consultant']['Contact'].to_list()


# Allocators that match on these specifically
EandF_Matching_pairs = [pair for pair in list(itertools.combinations(EandF_allocators, 2)) if pair in pair_count_list]
Pension_Matching_pairs = [pair for pair in list(itertools.combinations(Pension_allocators, 2)) if pair in pair_count_list]
OCIO_Matching_pairs = [pair for pair in list(itertools.combinations(OCIO_allocators, 2)) if pair in pair_count_list]
SFO_Matching_pairs = [pair for pair in list(itertools.combinations(SFO_allocators, 2)) if pair in pair_count_list]
MFO_Matching_pairs = [pair for pair in list(itertools.combinations(MFO_allocators, 2)) if pair in pair_count_list]
Hospital_Matching_pairs = [pair for pair in list(itertools.combinations(Hospital_allocators, 2)) if pair in pair_count_list]
Consultant_Matching_pairs = [pair for pair in list(itertools.combinations(Consultant_allocators, 2)) if pair in pair_count_list]

In [1073]:
allocator_data_db['AUM Range'] = allocator_data_db['AUM Range'].str.replace('>$10B',  '11').str.replace('$5B - $10B', '7').str.replace('>$20B','21').str.replace('>$1B','2')
allocator_data_db['AUM Range'] = allocator_data_db['AUM Range'].str.replace('Not Disclosed', '').str.replace('Not disclosed', '')
allocator_data_db.loc[allocator_data_db['AUM Range'] == '', 'AUM Range'] = None
allocator_data_db['AUM Range'] = allocator_data_db['AUM Range'].astype('float64')

In [1074]:
allocator_data_db.loc[allocator_data_db['AUM Range'] <=0.5, 'AUM Category'] = 'Under $0.5B'
allocator_data_db.loc[(allocator_data_db['AUM Range'] >0.5) & (allocator_data_db['AUM Range'] <= 2), 'AUM Category'] = '$0.5B - $2B' 
allocator_data_db.loc[(allocator_data_db['AUM Range'] >2) & (allocator_data_db['AUM Range'] <=5), 'AUM Category'] = '$2B - $5B'
allocator_data_db.loc[(allocator_data_db['AUM Range'] >5) & (allocator_data_db['AUM Range'] <=10), 'AUM Category'] = '$5B - $10B'
allocator_data_db.loc[(allocator_data_db['AUM Range'] >10) , 'AUM Category'] = 'Over $10B'

In [1075]:
EandF_AUM_Matching_pairs = []
Pension_AUM_Matching_pairs = []

for allocator1 in EandF_allocators:
    for allocator2 in EandF_allocators:
        if allocator1 != allocator2: 
            allocator1_AUM = allocator_data_db[allocator_data_db['Contact'] ==allocator1]['AUM Category'].values[0]
            allocator2_AUM = allocator_data_db[allocator_data_db['Contact'] ==allocator2]['AUM Category'].values[0]
            

            # if allocator1_AUM in ['Under $0.5B'] and  allocator2_AUM in ['Under $0.5B'] :
            #     EandF_AUM_Matching_pairs.append((allocator1, allocator2))

            # elif allocator1_AUM in [ '$0.5B - $2B'] and  allocator2_AUM in [ '$0.5B - $2B'] :
            #     EandF_AUM_Matching_pairs.append((allocator1, allocator2))

            if allocator1_AUM in ['Over $10B'] and  allocator2_AUM in ['Over $10B'] :
                EandF_AUM_Matching_pairs.append((allocator1, allocator2))

            # if allocator1_AUM in ['Under $0.5B', '$0.5B - $2B',  '$5B - $10B'] and  allocator2_AUM in['Under $0.5B', '$0.5B - $2B',  '$5B - $10B'] :
            #     EandF_AUM_Matching_pairs.append((allocator1, allocator2))
            # elif allocator1_AUM in ['$2B - $5B', '$5B - $10B', 'Over $10B'] and  allocator2_AUM in ['$2B - $5B', '$5B - $10B', 'Over $10B']:
            #     EandF_AUM_Matching_pairs.append((allocator1, allocator2))


for allocator1 in Pension_allocators:
    for allocator2 in Pension_allocators:
        if allocator1 != allocator2: 
            allocator1_AUM = allocator_data_db[allocator_data_db['Contact'] ==allocator1]['AUM Category'].values[0]
            allocator2_AUM = allocator_data_db[allocator_data_db['Contact'] ==allocator2]['AUM Category'].values[0]
            
            if allocator1_AUM in ['Under $0.5B', '$0.5B - $2B',  '$5B - $10B'] and  allocator2_AUM in['Under $0.5B', '$0.5B - $2B',  '$5B - $10B'] :
                Pension_AUM_Matching_pairs.append((allocator1, allocator2))
            elif allocator1_AUM in ['$2B - $5B', '$5B - $10B', 'Over $10B'] and  allocator2_AUM in ['$2B - $5B', '$5B - $10B', 'Over $10B']:
                Pension_AUM_Matching_pairs.append((allocator1, allocator2))

In [1078]:
MAonly_table_assignments = {}
MAonly_table_assignments['Session 4'] = meeting_assignments['Session 4']

##### **Check that the assignments are correct**
##### - F/ A ratios
##### - people seated in the correct groupings 


In [1080]:
qa_asset_theme_dict = {}

for session, tables in MAonly_table_assignments.items():

    print()
    updated_tables = {}
    
    for table, names in tables.items():

        allocator_names = [name for name in names if name.endswith('A')]
        fund_names = [name for name in names if name.endswith('F')]

        updated_allocator_names = [(allocator_data_db.loc[allocator_data_db['Contact'] == name, '*Description of Organization'].iloc[0] , allocator_data_db.loc[allocator_data_db['Contact'] == name, 'AUM Category'].iloc[0])
                         if  name in allocator_data_db['Contact'].values else name
                         for name in allocator_names]

        updated_fund_names = [fund_data_db.loc[fund_data_db['Contact'] == name, 'AUM Range'].iloc[0]
                         if  name in fund_data_db['Contact'].values else name
                         for name in fund_names]

        updated_tables[table] = updated_fund_names + updated_allocator_names
    
    qa_asset_theme_dict[session] = updated_tables

##### Which preferences were satisfied here? 
##### - No funds or allocators are sitting together so none are satisfied here

In [1081]:
# Create dictionary to store pair counts # 
MA_pair_counts = {}
for pair in pair_count_list:
    for table in MAonly_table_assignments['Session 4']:
        assignments = MAonly_table_assignments['Session 4'][table]
        if pair[0] in assignments and pair[1] in assignments:
            MA_pair_counts[pair] = 1

In [1082]:
MA_satisfied_fund_assetclass_pairs =list(set([pair for pair in fund_assetclass_matches if pair in MA_pair_counts  or (pair[1], pair[0]) in MA_pair_counts]))
MA_satisfied_fund_aum_pairs =  list(set([pair for pair in fund_aum_matches if pair in MA_pair_counts  or (pair[1], pair[0]) in MA_pair_counts]))

In [1083]:
if set(MA_pair_counts.values()) != {1}:
    raise ValueError()

In [1084]:
end_time = time.time()
print(f'This Took {round((end_time - start_time)/60, 2)} minutes from start to here')

This Took 38.67 minutes from start to here


# All Other Sessions
- removing table_session pairs, any relevant constraints, and objectives  this solves almost immediately 
- adding in table session pairs and the constriant that assigns the values to these variables this takes _ to solve 

## Set up LP Model


### Define LP Problem

In [1085]:
main_sessions = [session for session in [i for i in range(1, number_sessions+1)] if session != MAonly_session and session != lunch_session and session not in dive_deeper_sessions]

In [1086]:
main_table_assignments = {}
for session in main_sessions:
    main_table_assignments[f'Session {session}'] = meeting_assignments[f'Session {session}'].copy()

In [1088]:
# Create dictionary to store pair counts # 
main_pair_counts = {}
for pair in pair_count_list:
    main_pair_counts[pair] = 0
    for session in main_sessions: 
        for table in main_table_assignments[f'Session {session}']:
            assignments = main_table_assignments[f'Session {session}'][table]
            if pair[0] in assignments and pair[1] in assignments:
                main_pair_counts[pair] += 1
main_pair_counts = {k: v for k, v in main_pair_counts.items() if v>0}

In [1089]:
Counter(main_pair_counts.values())

Counter({1: 1440, 2: 220, 3: 2})

In [1090]:
main_satisfied_allocator_pref_pairs = list(set([pair for pair in allocator_pref_pairs if pair in main_pair_counts or (pair[1], pair[0]) in main_pair_counts]))
main_satisfied_fund_pref_pairs =list(set([pair for pair in fund_pref_pairs if pair in main_pair_counts or (pair[1], pair[0]) in main_pair_counts]))
main_satisfied_mutual_pref_pairs = list(set([pair for pair in mutual_pref_pairs if pair in main_pair_counts  or (pair[1], pair[0]) in main_pair_counts]))
main_satisfied_fund_assetclass_pairs = list(set([pair for pair in fund_assetclass_matches if pair in main_pair_counts  or (pair[1], pair[0]) in main_pair_counts]))
main_satisfied_fund_aum_pairs = list(set([pair for pair in fund_aum_matches if pair in main_pair_counts  or (pair[1], pair[0]) in main_pair_counts]))
main_satisfied_allocator_nopref_pairs = list(set([pair for pair in allocator_no_pref_pairs if pair in main_pair_counts  or (pair[1], pair[0]) in main_pair_counts]))

In [1091]:
print('LUNCH')
if  len(allocator_pref_pairs) > 0:
    print(round((len(set(lunch_satisfied_allocator_pref_pairs)) / len(allocator_pref_pairs) )*100, 2) , '% Allocator prefs satsified  in Lunch')
if  len(fund_pref_pairs) > 0:
    print(round((len(set(lunch_satisfied_fund_pref_pairs)) / len(fund_pref_pairs))*100, 2) , '% Fund prefs satsified in Lunch')
if  len(mutual_pref_pairs) > 0:
    print(round((len(set(lunch_satisfied_mutual_pref_pairs)) / len(mutual_pref_pairs))*100, 2) , '% Mutual prefs satsified in Lunch')
if  len(allocator_no_pref_pairs) > 0:
    print(round((len(set(lunch_satisfied_allocator_nopref_pairs)) / len(allocator_no_pref_pairs))*100, 2), '% No prefs satsified  in Lunch')

print('MAIN')
if  len(allocator_pref_pairs) > 0:
    print(round((len(set(main_satisfied_allocator_pref_pairs)) / len(allocator_pref_pairs) )*100, 2) , '% Allocator prefs satsified  in Main')
if  len(fund_pref_pairs) > 0:
    print(round((len(set(main_satisfied_fund_pref_pairs)) / len(fund_pref_pairs))*100, 2) , '% Fund prefs satsified in Main')
if  len(mutual_pref_pairs) > 0:
    print(round((len(set(main_satisfied_mutual_pref_pairs)) / len(mutual_pref_pairs))*100, 2) , '% Mutual prefs satsified in Main')
if  len(allocator_no_pref_pairs) > 0:
    print(round((len(set(main_satisfied_allocator_nopref_pairs)) / len(allocator_no_pref_pairs))*100, 2), '% No prefs satsified  in Main')

LUNCH
13.59 % Allocator prefs satsified  in Lunch
4.9 % Fund prefs satsified in Lunch
8.33 % Mutual prefs satsified in Lunch
0.0 % No prefs satsified  in Lunch
MAIN
60.52 % Allocator prefs satsified  in Main
50.98 % Fund prefs satsified in Main
91.67 % Mutual prefs satsified in Main
0.0 % No prefs satsified  in Main


# Reporting

##### Create dictionary with all values! 

### Look at pair counts


In [1093]:
pair_counts = {}

for pair in pair_count_list:
    pair_counts[pair] = 0 

    if pair in lunch_pair_counts:
        pair_counts[pair] += lunch_pair_counts[pair]
    if pair in MA_pair_counts:
        pair_counts[pair] += MA_pair_counts[pair]
    if pair in main_pair_counts:
        pair_counts[pair] += main_pair_counts[pair]

In [1095]:
Counter(pair_counts.values())

# print(f'{round((Counter(pair_counts.values())[1]*100 / Counter(pair_counts.values())[1]),1)}% of pairs sat together 1 time')
# print(f'{round((Counter(pair_counts.values())[2]*100 / Counter(pair_counts.values())[1]),2)}% of pairs sat together 2 times')
# print(f'{round((Counter(pair_counts.values())[3]*100 / Counter(pair_counts.values())[1]),3)}% of pairs sat together 3 times')
# print(f'{round((Counter(pair_counts.values())[4]*100 / Counter(pair_counts.values())[1]),4)}% of pairs sat together 4 times')

Counter({0: 5760, 1: 1699, 2: 285, 3: 6})

##### Quality Check for pair counts

##### Sort the pair_counts so that they match up with the pair_count_list which creates all the table_session_pair_variables

In [1096]:
pair_counts_sorted = {}
for pair in pair_counts:
    # if the inverse of the pair is in pair counts list swap them 
    if pair not in pair_count_list:
        if (pair[1], pair[0]) in pair_count_list:
            pair_counts_sorted[(pair[1], pair[0])] = pair_counts[pair]
    elif pair in pair_count_list:
        pair_counts_sorted[pair] = pair_counts[pair]

pair_counts = pair_counts_sorted

# sys.exit()

##### Check that the number of Funds and Allocators are correct for each
- If youre looking at an old rerun it may be incorrect since the updated data determines the table frameworks 

this might throw errors if the lunch session was run as a main session  

In [1099]:
for session in [i for i in range(1, number_sessions+1) if i!= lunch_session]:

    for table in range(1, number_tables_dict[session] + 1):
        assignments = meeting_assignments[f'Session {session}'][table]
        # print(assignments)
        if lunch_session != [] and session == lunch_session:
            print(assignments)
            assignments = [value for value in list(assignments.values() ) if value != ()]
    
        assignments =[ item for item in assignments if not item.endswith('_S')]
        framework = table_frameworks[f'Session {session}'][f'Table {table}']
        if framework.count('A') != sum(1 for item in assignments if item.endswith('_A')):
            print('Session', session,'Table', table, 'Allocators')
            print('Table Framework Count A:', (framework.count('A')),'Results Count A:', (sum(1 for item in assignments if item.endswith('_A'))) )
        if framework.count('F') != sum(1 for item in assignments if item.endswith('_F')):
            print('Session', session,'Table', table, 'Funds')
            print('Table Framework Count F:', (framework.count('F')),'Results Count F:', (sum(1 for item in assignments if item.endswith('_F'))) )
        if framework.count('F') == sum(1 for item in assignments if item.endswith('_F')) and framework.count('A') == sum(1 for item in assignments if item.endswith('_A')):
            pass

##### Are people seated more than one time in a session?


In [1100]:

def convert_session_to_list_of_lists(session_data):
    result = []
    for key, inner_dict in session_data.items():
        names = [name for name in inner_dict.values() if isinstance(name, str)]
        result.append(names)
    return result


# Create a Pandas Series
for session in range(1, number_sessions+1):
    if lunch_session != [] and session == lunch_session: 
        df = pd.DataFrame(pd.Series([value for sublist in convert_session_to_list_of_lists(meeting_assignments[f'Session {session}']) for value in sublist]).value_counts()).reset_index()

    elif session != lunch_session:
        df = pd.DataFrame(pd.Series([value for sublist in meeting_assignments[f'Session {session}'].values() for value in sublist]).value_counts()).reset_index()
    df = df[df['index']!= 'Empty Seat']
    if df[df['count'] >1].shape[0] >0:
        to_print = df[df['count'] >1].shape[0]
        raise ValueError(f'There are {to_print} names in Session {session} that are seated twice')
    #     print(f'There are {(df[df['count'] >1].shape[0])} names in Session {session} that are seated twice')

In [1101]:
print('The list of names in session 1 is the same as the set of the list of names in session 1 (duplicates if false):',
       len(set([value for sublist in meeting_assignments['Session 1'].values() for value in sublist])) == len(([value for sublist in meeting_assignments['Session 1'].values() for value in sublist])))
print(f'Length of the set: {len(set([value for sublist in meeting_assignments['Session 1'].values() for value in sublist]))}\nLength of the list: {len(([value for sublist in meeting_assignments['Session 1'].values() for value in sublist]))}')

The list of names in session 1 is the same as the set of the list of names in session 1 (duplicates if false): True
Length of the set: 145
Length of the list: 145


##### Quality Check for Fully Seated tables

In [1102]:
# QA that all seats are filled
qa_count = 0 
for i in range(1,number_sessions+1):
    session = meeting_assignments[f'Session {i}']
    for table in session: 
        assignments = meeting_assignments[f'Session {i}'][table]
        if i == lunch_session:
            assignments = [value for value in list(assignments.values() ) if value != ()]
        if assignments.count('A') > 0 or assignments.count('F') > 0:
            qa_count += 1
            print(f"Session {i} Table {table} not fully seated!")

if qa_count > 0 :
    raise ValueError('Tables are not fully seated')
else:
    print('All tables are fully seated :)')

All tables are fully seated :)


### Create Pair Counts dfs

All sessions

In [1103]:
counts_df = pd.DataFrame.from_dict(Counter(pair_counts.values()), orient='index', columns=['Count']).reset_index()
counts_df.columns = ['# Times Sat Together', '# Occurrences']
counts_df= counts_df.sort_values(by ='# Times Sat Together')
counts_df['% of Total Occurrences'] = round(((counts_df['# Occurrences'] / (counts_df['# Occurrences'].sum())) * 100),2)

# Convert the dictionary to a list of dictionaries
data_list = [{'Person 1': pair[0], 'Person 2': pair[1], 'Number Times sat together': value} for pair, value in pair_counts.items()]

# Create a DataFrame from the list
pair_counts_matrix = pd.DataFrame(data_list)

# Format back to Correct Name (not contact format)
for col in ['Person 1', 'Person 2']:
    # remove underscore replace with space
    pair_counts_matrix[col] = pair_counts_matrix[col].str.replace('_', ' ')
    #Captialize
    pair_counts_matrix[col] = pair_counts_matrix[col].str.title()

    # replace " F" with "(F)" and ' A' with (A) in all columns # use positive lookback assertion
    pair_counts_matrix = pair_counts_matrix.replace({'(?<= )F$': ' (F)'}, regex=True)
    pair_counts_matrix = pair_counts_matrix.replace({'(?<= )A$': ' (A)'}, regex=True)

without lunch session (8)

In [1104]:
pair_counts_no8 = {}

for pair in pair_count_list:

    pair_counts_no8[pair] = 0 
    if pair in MA_pair_counts:
        pair_counts_no8[pair] += MA_pair_counts[pair]
    if pair in main_pair_counts:
        pair_counts_no8[pair] += main_pair_counts[pair]


counts_no8_df = pd.DataFrame.from_dict(Counter(pair_counts_no8.values()), orient='index', columns=['Count']).reset_index()
counts_no8_df.columns = ['# Times Sat Together', '# Occurrences']
counts_no8_df= counts_df.sort_values(by ='# Times Sat Together')
counts_no8_df['% of Total Occurrences'] = round(((counts_no8_df['# Occurrences'] / (counts_no8_df['# Occurrences'].sum())) * 100),2)

# Convert the dictionary to a list of dictionaries
data_list = [{'Person 1': pair[0], 'Person 2': pair[1], 'Number Times sat together': value} for pair, value in pair_counts_no8.items()]

# Create a DataFrame from the list
pair_counts_matrix_no8 = pd.DataFrame(data_list)


# Format back to Correct Name (not contact format)
for col in ['Person 1', 'Person 2']:
    # remove underscore replace with space
    pair_counts_matrix_no8[col] = pair_counts_matrix_no8[col].str.replace('_', ' ')
    #Captialize
    pair_counts_matrix_no8[col] = pair_counts_matrix_no8[col].str.title()

    # replace " F" with "(F)" and ' A' with (A) in all columns # use positive lookback assertion
    pair_counts_matrix_no8 = pair_counts_matrix_no8.replace({'(?<= )F$': ' (F)'}, regex=True)
    pair_counts_matrix_no8 = pair_counts_matrix_no8.replace({'(?<= )A$': ' (A)'}, regex=True)

Just main sessions 

In [1105]:
counts_main_df = pd.DataFrame.from_dict(Counter(main_pair_counts.values()), orient='index', columns=['Count']).reset_index()
counts_main_df.columns = ['# Times Sat Together', '# Occurrences']
counts_main_df= counts_df.sort_values(by ='# Times Sat Together')
counts_main_df['% of Total Occurrences'] = round(((counts_main_df['# Occurrences'] / (counts_main_df['# Occurrences'].sum())) * 100),2)

# Convert the dictionary to a list of dictionaries
data_list = [{'Person 1': pair[0], 'Person 2': pair[1], 'Number Times sat together': value} for pair, value in main_pair_counts.items()]

# Create a DataFrame from the list
pair_counts_matrix_main = pd.DataFrame(data_list)


# Format back to Correct Name (not contact format)
for col in ['Person 1', 'Person 2']:
    # remove underscore replace with space
    pair_counts_matrix_main[col] = pair_counts_matrix_main[col].str.replace('_', ' ')
    #Captialize
    pair_counts_matrix_main[col] = pair_counts_matrix_main[col].str.title()

    # replace " F" with "(F)" and ' A' with (A) in all columns # use positive lookback assertion
    pair_counts_matrix_main = pair_counts_matrix_main.replace({'(?<= )F$': ' (F)'}, regex=True)
    pair_counts_matrix_main = pair_counts_matrix_main.replace({'(?<= )A$': ' (A)'}, regex=True)

##### Pad the tables and the seats with NA values so that the rest of the code can work: 

In [1106]:
## Change all tables to # instead of Tablce #

for session in meeting_assignments:
    
    if lunch_session != [] and session != lunch_session: 
        updated_dict = {}
        for key, value in meeting_assignments[session].items():
            if not isinstance(key, int):
                new_key = int(key.split(' ')[-1])
                updated_dict[new_key] = value
            else:
                updated_dict[key] = value
        meeting_assignments[session] = updated_dict

Change to tuples rather than lists 

In [1107]:
for session in range(1, number_sessions+1):
        for table in range(1, number_tables_dict[session]+1):
            if lunch_session != [] and session != lunch_session:
                assignments = meeting_assignments[f'Session {session}'][table]
                meeting_assignments[f'Session {session}'][table] = tuple(assignments)

In [1108]:
# Pad sessions with empty tables 

# Find the highest number of tables 
# max_session = max(len(meeting_assignments[f'Session {i}']) for session_tables in meeting_assignments.values())
max_session = max(number_tables_dict.values())

for i in range (1, number_sessions+1): # meeting assignments
    if lunch_session != [] and i != lunch_session:
        while len(meeting_assignments[f'Session {i}']) < max_session:
            last_table = max(meeting_assignments[f'Session {i}'].keys())
            new_table_name = last_table + 1
            meeting_assignments[f'Session {i}'][new_table_name] = ()
    else:
        while len(meeting_assignments[f'Session {i}']) < max_session:
            last_table = max(meeting_assignments[f'Session {i}'].keys())
            new_table_name = last_table + 1
            meeting_assignments[f'Session {i}'][new_table_name] = {}

# Pad tables with empty seats
for i in range(1, number_sessions+1): # meeting assignments 
    if lunch_session != [] and i != lunch_session:
        tables = meeting_assignments[f'Session {i}']
        for table_key, table in tables.items():
            while len(table) < max_table_size_dict[i]:
                table = table + ('Empty Seat',)
            tables[table_key] = table  # update table in dictionary
        meeting_assignments[f'Session {i}'] = tables  # update tables for the session

In [1109]:
# Pad sessions with empty seats 

# Iterate through the dictionary
for session, inner_dict in meeting_assignments.items():
    updated_inner_dict = {}  # Create a new inner dictionary for the updated values
    
    for inner_key, inner_value in inner_dict.items():
        if isinstance(inner_key, str) and inner_key.startswith('Table '):
            # If the inner key starts with 'Table ', extract the integer part
            updated_inner_key = int(inner_key.split(' ')[1])
        else:
            updated_inner_key = inner_key  # Keep the key as is
        
        updated_inner_dict[updated_inner_key] = inner_value  # Update the inner dictionary
    
    meeting_assignments[session] = updated_inner_dict  # Update the outer dictionary


## Add dictionary for empty tables in session unch

if lunch_session != [] :
    for table in  meeting_assignments[f'Session {lunch_session}']:
        if meeting_assignments[f'Session {lunch_session}'][table] == {}:
            meeting_assignments[f'Session {lunch_session}'][table] = {i: () for i in range(1, 10 + 1)}


# Lunch session empty seats 
if lunch_session != [] :
    for table in meeting_assignments[f'Session {lunch_session}']:
        meeting_assignments[f'Session {lunch_session}'][table] = {key: 'Empty Seat' if value == () else value for key, value in meeting_assignments[f'Session {lunch_session}'][table].items()}

In [ ]:
# sys.exit()

### Create Meeting Assignments Dataframe 

## Match the formatting of all of them with the formatting of lunch!! 

Remove lunch from meeting assignments - I want to create that df sep and then glue it on 

In [1110]:
meeting_assignments_lunch = meeting_assignments['Session 8']
meeting_assignments_main =  {key: value for key, value in meeting_assignments.items() if key != 'Session 8'}

No clue why I'm geting extra seats but here we are with a fix ...

In [1111]:
for session in range(1, 8):

    for table in [19,20]:
        if len(meeting_assignments_main[f'Session {session}'][table]) >10:
            print(session)
            print(len(meeting_assignments_main[f'Session {session}'][table]))
            print(meeting_assignments_main[f'Session {session}'][table])
            if  set(meeting_assignments_main[f'Session {session}'][table]) == {'Empty Seat'}:
                meeting_assignments_main[f'Session {session}'][table] = tuple(['Empty Seat' for i in range(1, max_table_size_dict[session])])
            # raise ValueError()

In [ ]:
sys.exit()

In [1136]:
for session in range (1, 8):
    for table in range(1, number_tables_dict[session]+1):
        assignments = list(meeting_assignments_main[f'Session {session}'][table])
        print(len(assignments))
        if len(assignments) == 9:
            assignments.append('Empty Seat')
            meeting_assignments_main[f'Session {session}'][table] = tuple(assignments)
        print(len(assignments))

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10


Format the Main sessions 

In [1137]:
meeting_assignments_main_df = pd.DataFrame.from_dict({(i, j): meeting_assignments_main[i][j] for i in meeting_assignments_main.keys() for j in meeting_assignments_main[i].keys()},
                            orient='index')

# Name Sessions 
sessions_range = [f'Session {i}' for i in [i for i in range(1, number_sessions) if i != lunch_session] for _ in range(max(number_tables_dict.values()) )]
meeting_assignments_main_df['Session'] = sessions_range

table_range = []

for i in [i for i in range(1, number_sessions+1) if i != lunch_session]:
    for j in range(1, max(number_tables_dict.values()) + 1):
        table_range.append(j)
        
# Name Tables 
meeting_assignments_main_df['Table'] = table_range

# Set index to session and table 
meeting_assignments_main_df.set_index(['Session','Table'], inplace = True)

# Replace _ with space
meeting_assignments_main_df = meeting_assignments_main_df.applymap(        
    lambda x: x.replace('_', ' ') if isinstance(x, str) else x)

# Capitalize Names 
meeting_assignments_main_df = meeting_assignments_main_df.applymap(lambda x: x.title() if isinstance(x, str) else x)

##### Format To Match IR Output

In [1138]:
# replace " F" with "(F)" in all columns # use positive lookback assertion
meeting_assignments_main_df = meeting_assignments_main_df.replace({'(?<= )F$': ' (F)'}, regex=True)
meeting_assignments_main_df = meeting_assignments_main_df.replace({'(?<= )A$': ' (A)'}, regex=True)
meeting_assignments_main_df = meeting_assignments_main_df.replace({'(?<= )S$': ' (S)'}, regex=True)
# Replace "Empty Seat" with Z so that it goes to the end 
meeting_assignments_main_df= meeting_assignments_main_df.replace('Empty Seat', '(Z)')



meeting_assignments_main_df = meeting_assignments_main_df.T.stack().reset_index()
meeting_assignments_main_df = meeting_assignments_main_df.drop(columns = ['level_0']).sort_values(by  = ["Table"])
meeting_assignments_main_df = meeting_assignments_main_df.set_index('Table')


##### Sort by Table and format empty seats

In [1139]:
# Sorts the (A) and (F) in the dataframe
def sort_group_by_marker(group):
    group[['col1', 'col2']] = group[['Session 2', 'Session 4']].apply(lambda x: x.str[-3:])
    group = group.sort_values(by=['col1', 'col2'], ascending=True)
    group = group.drop(['col1', 'col2'], axis=1)
    return group


meeting_assignments_main_df = meeting_assignments_main_df.reset_index()

# apply sorting function to each group within 'Table' column
meeting_assignments_main_df = meeting_assignments_main_df.groupby('Table', group_keys=False).apply(sort_group_by_marker)

# reset index and drop group index level#
meeting_assignments_main_df= meeting_assignments_main_df.set_index('Table')
meeting_assignments_main_df= meeting_assignments_main_df.replace('(Z)', '       -       ')

meeting_assignments_main_df = meeting_assignments_main_df.fillna( '       -       ')

### Format the Lunch DF

In [1140]:
meeting_assignments_lunch_df = pd.DataFrame.from_dict({(i, j): meeting_assignments_lunch[i][j] for i in meeting_assignments_lunch.keys() for j in meeting_assignments_lunch[i].keys()},
                            orient='index')

# Name Sessions 
meeting_assignments_lunch_df['Session'] = lunch_session

table_range = [i for i in range(1, number_tables_dict[lunch_session]+1)]

meeting_assignments_lunch_df = meeting_assignments_lunch_df.reset_index().rename(columns = {'index':'(table,seat)', 0:f'Session {lunch_session}'})

meeting_assignments_lunch_df['Table'] = meeting_assignments_lunch_df['(table,seat)'].apply(lambda x: x[0])
meeting_assignments_lunch_df['Seat'] = meeting_assignments_lunch_df['(table,seat)'].apply(lambda x: x[1])

meeting_assignments_lunch_df = meeting_assignments_lunch_df.drop(columns =['(table,seat)', 'Session'] )


# Replace _ with space
meeting_assignments_lunch_df = meeting_assignments_lunch_df.applymap(        
    lambda x: x.replace('_', ' ') if isinstance(x, str) else x)

# Capitalize Names 
meeting_assignments_lunch_df = meeting_assignments_lunch_df.applymap(lambda x: x.title() if isinstance(x, str) else x)


# replace " F" with "(F)" in all columns # use positive lookback assertion
meeting_assignments_lunch_df = meeting_assignments_lunch_df.replace({'(?<= )F$': ' (F)'}, regex=True)
meeting_assignments_lunch_df = meeting_assignments_lunch_df.replace({'(?<= )A$': ' (A)'}, regex=True)
meeting_assignments_lunch_df = meeting_assignments_lunch_df.replace({'(?<= )S$': ' (S)'}, regex=True)
# Replace "Empty Seat" with Z so that it goes to the end 
meeting_assignments_lunch_df= meeting_assignments_lunch_df.replace('Empty Seat', '(Z)')

meeting_assignments_lunch_df= meeting_assignments_lunch_df.replace('(Z)', '       -       ')
meeting_assignments_lunch_df = meeting_assignments_lunch_df.fillna( '       -       ')

meeting_assignments_lunch_df = meeting_assignments_lunch_df.set_index(['Table'])

In [1142]:
meeting_assignments_df = pd.concat([meeting_assignments_main_df, meeting_assignments_lunch_df.drop(columns= 'Seat')], axis=1)

##### **Use this if you need it to search for a case insensitive name**

In [ ]:
# Search for an element starting with "lisa" (case-insensitive)
pattern = re.compile(r'^susan_chen', re.IGNORECASE)
matching_elements = [element for element in funds if pattern.match(element)]

### Create Preferences dataframes

#### **Raw Preferences by F/A**

In [1143]:
raw_prefs_for_funds_dict = {}

for fund in funds:
    prefs_for_fund = []
    for allocator in allocators:

        if allocator_prefs_dict[allocator] != None and fund in allocator_prefs_dict[allocator]:
            prefs_for_fund.append(allocator)
        
    raw_prefs_for_funds_dict[fund] = prefs_for_fund


raw_prefs_for_allocators_dict = {}
for allocator in allocators:
    prefs_for_allocator = []
    for fund in funds:
        if fund_prefs_dict[fund] != None and  allocator in fund_prefs_dict[fund] :
            prefs_for_allocator.append(fund)
    raw_prefs_for_allocators_dict[allocator] = prefs_for_allocator

In [1144]:
# Pad with NA values
max_length = max(len(value) for value in raw_prefs_for_funds_dict.values())

for fund in raw_prefs_for_funds_dict:  
    number_padded_values = (max_length - len(raw_prefs_for_funds_dict[fund]))
    raw_prefs_for_funds_dict[fund].extend([' ' for i in range(1,number_padded_values+1)])

max_length = max(len(value) for value in raw_prefs_for_allocators_dict.values())

for allocator in raw_prefs_for_allocators_dict:  
    number_padded_values = (max_length - len(raw_prefs_for_allocators_dict[allocator]))
    raw_prefs_for_allocators_dict[allocator].extend([' ' for i in range(1,number_padded_values+1)])

# Revert from Contact Form
raw_prefs_for_funds_dict = {key[:-2].replace('_', ' ').title(): value for key, value in raw_prefs_for_funds_dict.items()}
raw_prefs_for_funds_dict = {key: [value[:-2].replace('_', ' ').title() for value in values] for key, values in raw_prefs_for_funds_dict.items()}

raw_prefs_for_allocators_dict = {key[:-2].replace('_', ' ').title(): value for key, value in raw_prefs_for_allocators_dict.items()}
raw_prefs_for_allocators_dict = {key: [value[:-2].replace('_', ' ').title() for value in values] for key, values in raw_prefs_for_allocators_dict.items()}

# Create Dfs 
raw_prefs_A_df = pd.DataFrame(raw_prefs_for_allocators_dict).T
raw_prefs_F_df = pd.DataFrame(raw_prefs_for_funds_dict).T

# Add column names 
raw_prefs_A_df.columns =  [f'Manager {i+1}' for i in range(len(raw_prefs_A_df.columns))]
raw_prefs_F_df.columns =  [f'Allocator {i+1}' for i in range(len(raw_prefs_F_df.columns))]



##### **Satisfied_prefs list** 

In [1145]:
satisfied_mutual_prefs = main_satisfied_mutual_pref_pairs + lunch_satisfied_mutual_pref_pairs 
satisfied_mutual_prefs = list(set(satisfied_mutual_prefs))

satisfied_A_prefs = list(set(main_satisfied_allocator_pref_pairs + lunch_satisfied_allocator_pref_pairs ))

satisfied_F_prefs = list(set(main_satisfied_fund_pref_pairs + lunch_satisfied_fund_pref_pairs ))

allocator_no_pref_pairs_sat_together =list(set(lunch_satisfied_allocator_nopref_pairs + main_satisfied_allocator_nopref_pairs))

In [1146]:
print(f"{round(((len(satisfied_A_prefs)/len(allocator_pref_pairs))*100),2)} % Hit Rate for Allocator Preferences")
print(f"{round(((len(satisfied_F_prefs)/len(fund_pref_pairs))*100),2)} % Hit Rate for Fund Preferences")
print(f"{round(((len(satisfied_mutual_prefs)/len(mutual_pref_pairs))*100),2)} % Hit Rate for Mutual Preferences")
print(f"{round(((len(allocator_no_pref_pairs_sat_together)/len(allocator_no_pref_pairs))*100),2)} % Hit Rate for Allocator NO Preferences")
print('     Want this ^ to be as low as possible')

71.52 % Hit Rate for Allocator Preferences
55.88 % Hit Rate for Fund Preferences
100.0 % Hit Rate for Mutual Preferences
0.0 % Hit Rate for Allocator NO Preferences
     Want this ^ to be as low as possible


In [1147]:
# Calculate hit rates
hit_rate_allocator = round((len(satisfied_A_prefs) / len(allocator_pref_pairs)) * 100, 2)
hit_rate_fund = round((len(satisfied_F_prefs) / len(fund_pref_pairs)) * 100, 2)
hit_rate_mutual = round((len(satisfied_mutual_prefs) / len(mutual_pref_pairs)) * 100, 2)
hit_rate_allocator_no_pref = round((len(allocator_no_pref_pairs_sat_together) / len(allocator_no_pref_pairs)) * 100, 2)

# Create DataFrame
data = {
    "Hit Rate for Allocator Prefs": [hit_rate_allocator],
    "Hit Rate for Fund Prefs": [hit_rate_fund],
    "Hit Rate for Mutual Prefs": [hit_rate_mutual],
    "Hit Rate for Allocator NO Prefs": [hit_rate_allocator_no_pref],
}

summary_df = pd.DataFrame(data)

summary_df =summary_df.T.reset_index().rename(columns = {'index':'Summary Stat', 0: 'Value'})


In [ ]:
# sys.exit()

#####  **satisfied_prefs dictionary** 

In [1148]:
# Initialize an empty dictionary
satisfied_prefs = {}

# Add satisfied fund preferences to the dictionary 
for F_pref, A_pref in satisfied_F_prefs:
    # Check if the key (ending in 'F') is already in the dictionary
    if F_pref in satisfied_prefs:
        satisfied_prefs[F_pref].append(A_pref)
    else:
        # If the key is not in the dictionary, create a new list with the 'A' preference
        satisfied_prefs[F_pref] = [A_pref]

# Add satisfied allocator preferences to the dictionary 
for F_pref, A_pref in satisfied_A_prefs:
    # Check if the key (ending in 'A') is already in the dictionary
    if A_pref in satisfied_prefs:
        satisfied_prefs[A_pref].append(F_pref)
    else:
        # If the key is not in the dictionary, create a new list with the 'A' preference
        satisfied_prefs[A_pref] = [F_pref]

#### Add the people who didnt have prefs or that had none satisfied 

In [1149]:
for contact in attendees:
    if contact not in satisfied_prefs:
        satisfied_prefs[contact] = []

##### Create dictionaries for companies

In [1150]:
# Create Contact tab in the fund and allocator database data 
fund_df_db['Contact'] =fund_df_db['FirstName'].str.lower() + '_' + fund_df_db['LastName'].str.lower() + '_F'
allocator_df_db['Contact'] =allocator_df_db['FirstName'].str.lower() + '_' + allocator_df_db['LastName'].str.lower() + '_A'

# Create Company dictionaries
fund_company_dict = dict(zip(fund_df_db['Contact'].tolist(), fund_df_db['Company'].tolist()))
allocator_company_dict = dict(zip(allocator_df_db['Contact'].tolist(), allocator_df_db['Company'].tolist()))

##### **unsatisfied_prefs list** 

In [1151]:
unsatisfied_F_prefs = []
for pair in fund_pref_pairs:
    if pair not in satisfied_F_prefs:
        unsatisfied_F_prefs.append(pair)

if len(set(unsatisfied_F_prefs + satisfied_F_prefs))!= len(set(fund_pref_pairs)):
    raise ValueError('Satisfied prefs + Unsatisfied Prefs != all prefs')


unsatisfied_A_prefs = []
for pair in allocator_pref_pairs:
    if pair not in satisfied_A_prefs:
        unsatisfied_A_prefs.append(pair)
if len(set(unsatisfied_A_prefs)) + len((set(satisfied_A_prefs))) != len(set(allocator_pref_pairs)):
    raise ValueError('Satisfied prefs + Unsatisfied Prefs != all prefs')

In [1152]:
prefs_dict = {**fund_prefs_dict, **allocator_prefs_dict}

In [1153]:
unsatisfied_prefs= {}

for key in prefs_dict:
    if key in satisfied_prefs and key in prefs_dict:
        satisfied_values = satisfied_prefs[key]
        raw_values = prefs_dict[key]

        if raw_values != None:
            unsatisfied_values = [value for value in raw_values if value not in satisfied_values]
            unsatisfied_prefs[key] = unsatisfied_values
        else:
            unsatisfied_prefs[key] = []

#### **Raw Preferences DF**

In [ ]:
# for key, value in fund_prefs_dict.items():
#     if value is None:
#         fund_prefs_dict[key] = []

# for key, value in allocator_prefs_dict.items():
#     if value is None:
#         allocator_prefs_dict[key] = []

In [ ]:
# raw_prefs_df_F = pd.DataFrame.from_dict(fund_prefs_dict, orient='index').reset_index()
# raw_prefs_df_A = pd.DataFrame.from_dict(allocator_prefs_dict, orient='index').reset_index()

# raw_prefs_df = pd.concat([raw_prefs_df_F, raw_prefs_df_A])


# # Variablize how many satisfied pref columns there are based on the max 
# max_length = (max(len(values) for values in allocator_prefs_dict.values()))
# raw_prefs_df.columns =  ['Name']+ [f'Preference {i}' for i in range(1, max_length + 1)]

# raw_prefs_df['Role'] = raw_prefs_df['Name'].apply(lambda x: 'Manager' if x.endswith('F') else 'Allocator')

# # Format Dataframe 
# # Replace _ with space
# raw_prefs_df = raw_prefs_df.applymap(        
#     lambda x: x.replace('_', ' ') if isinstance(x, str) else x)

# # Capitalize Names 
# raw_prefs_df= raw_prefs_df.applymap(lambda x: x.title() if isinstance(x, str) else x)

# raw_prefs_df = raw_prefs_df.reindex(columns=['Name'] + ['Role'] + ['Company'] + [f'Preference {i}' for i in range(1, max_length + 1)])
# # Remove F and A 
# raw_prefs_df = raw_prefs_df.applymap(lambda x: x[:-2] if isinstance(x, str) and (x.endswith(" F") or x.endswith(" A")) else x)
# # Change None to Empty
# raw_prefs_df = raw_prefs_df.fillna('')

##### **Satisfied prefs dataframe**

In [1154]:
# satisfied preferences as dataframe
satisfied_prefs_df = pd.DataFrame.from_dict(satisfied_prefs, orient='index').reset_index()

# Variablize how many satisfied pref columns there are based on the max 
max_length = (max(len(values) for values in satisfied_prefs.values()))
satisfied_prefs_df.columns =  ['Name']+ [f'Satisfied Pref {i}' for i in range(1, max_length + 1)]

satisfied_prefs_df['Role'] = satisfied_prefs_df['Name'].apply(lambda x: 'Manager' if x.endswith('F') else 'Allocator')


# Map allocator companies
satisfied_prefs_df['Company'] = satisfied_prefs_df['Name'].map(allocator_company_dict)
# Map the values that are Nan 
satisfied_prefs_df.loc[satisfied_prefs_df['Company'].isna(), 'Company'] = satisfied_prefs_df.loc[satisfied_prefs_df['Company'].isna(), 'Name'].map(fund_company_dict)

# Format Dataframe 
# Replace _ with space
satisfied_prefs_df = satisfied_prefs_df.applymap(        
    lambda x: x.replace('_', ' ') if isinstance(x, str) else x)


# Format Dataframe 
# Replace _ with space
satisfied_prefs_df = satisfied_prefs_df.applymap(        
    lambda x: x.replace('_', ' ') if isinstance(x, str) else x)

# Capitalize Names 
satisfied_prefs_df= satisfied_prefs_df.applymap(lambda x: x.title() if isinstance(x, str) else x)

satisfied_prefs_df = satisfied_prefs_df.reindex(columns=['Name'] + ['Role'] + ['Company'] + [f'Satisfied Pref {i}' for i in range(1, max_length + 1)])
# Remove F and A 
satisfied_prefs_df = satisfied_prefs_df.applymap(lambda x: x[:-2] if isinstance(x, str) and (x.endswith(" F") or x.endswith(" A")) else x)
# Change None to Empty
satisfied_prefs_df = satisfied_prefs_df.fillna('')

# # Group by the Role and then Sort satisfied prefs
# satisfied_prefs_df =  satisfied_prefs_df.groupby('Role', 'Company'
#                                         ).apply(
#     lambda x: x.sort_values(['Satisfied Pref 2'], ascending=False))

In [1155]:
## Add the % satisfied 
satisfied_prefs_df['% Satisfied'] = np.nan

for index, row in satisfied_prefs_df.iterrows():


    if row['Role'] == 'Manager':
        fund = row['Name'].lower().replace(' ', '_') + '_F'
        if fund =='colter_van_domelen_F': fund = 'colter_van domelen_F'
        if fund_prefs_dict[fund] != None:
            satisfied_prefs_df.loc[index, '% Satisfied'] = (len(satisfied_prefs[fund]) / len(fund_prefs_dict[fund]))
        else:
            satisfied_prefs_df.loc[index, '% Satisfied'] = 1
    elif row['Role'] == 'Allocator':
        allocator = row['Name'].lower().replace(' ', '_') + '_A'
        if allocator_prefs_dict[allocator] != None:
            satisfied_prefs_df.loc[index, '% Satisfied'] = (len(satisfied_prefs[allocator]) / len(allocator_prefs_dict[allocator]))
        else:
            satisfied_prefs_df.loc[index, '% Satisfied'] = 1


In [ ]:
# satisfied_prefs_df.to_excel(r"C:\Users\Administrator\Downloads\SeatingArrangement_Test.xlsx", sheet_name='SatisfiedPrefs', index=True)


In [1156]:
satisfied_prefs_df['Contact'] = satisfied_prefs_df['Name'].str.lower().str.replace(' ', '_')

satisfied_prefs_df.loc[satisfied_prefs_df['Role'] == 'Manager', 'Contact'] = satisfied_prefs_df['Contact'] + '_F'
satisfied_prefs_df.loc[satisfied_prefs_df['Role'] == 'Allocator', 'Contact'] = satisfied_prefs_df['Contact'] + '_A'


funds_no_satisfied_prefs = [name for name in satisfied_prefs if satisfied_prefs[name] == [] and name.endswith('F')]
funds_no_satisfied_prefs = [name for name in funds_no_satisfied_prefs if name not in [name for name in fund_prefs_dict if fund_prefs_dict[name] == None]]

allocators_no_satisfied_prefs = [name for name in satisfied_prefs if satisfied_prefs[name] == [] and name.endswith('A')]
allocators_no_satisfied_prefs = [name for name in allocators_no_satisfied_prefs if name not in [name for name in allocator_prefs_dict if allocator_prefs_dict[name] == None]]

if not (allocator_data_db[allocator_data_db['Contact'].isin(allocators_no_satisfied_prefs)]['Preference'].unique() == [None]).all():
    print("Some allocators with prefs have non satisfied")

if not (fund_data_db[fund_data_db['Contact'].isin(funds_no_satisfied_prefs)]['Preference'].unique() == [None]).all():
    print("Some funds with prefs have non satisfied")

Some allocators with prefs have non satisfied
Some funds with prefs have non satisfied


##### **Unsatisfied prefs dataframe**

In [1157]:
# satisfied preferences as dataframe
unsatisfied_prefs_df = pd.DataFrame.from_dict(unsatisfied_prefs, orient='index').reset_index()

# Variablize how many satisfied pref columns there are based on the max 
max_length = (max(len(values) for values in unsatisfied_prefs.values()))
unsatisfied_prefs_df.columns =  ['Name']+ [f'Unsatisfied Pref {i}' for i in range(1, max_length + 1)]

# Add role columns
unsatisfied_prefs_df['Role'] = unsatisfied_prefs_df['Name'].apply(lambda x: 'Manager' if x.endswith('F') else 'Allocator')

# Map allocator companies
unsatisfied_prefs_df['Company'] = unsatisfied_prefs_df['Name'].map(allocator_company_dict)
# Map the values that are Nan 
unsatisfied_prefs_df.loc[unsatisfied_prefs_df['Company'].isna(), 'Company'] = unsatisfied_prefs_df.loc[unsatisfied_prefs_df['Company'].isna(), 'Name'].map(fund_company_dict)


if not set(unsatisfied_prefs_df['Name'].unique()) == set(attendees):
    raise ValueError('not all attendees in this df!')

# Format Dataframe 
# Replace _ with space
unsatisfied_prefs_df = unsatisfied_prefs_df.applymap(        
    lambda x: x.replace('_', ' ') if isinstance(x, str) else x)


unsatisfied_prefs_df = unsatisfied_prefs_df.reindex(columns=['Name'] + ['Role'] + ['Company'] + [f'Unsatisfied Pref {i}' for i in range(1, max_length + 1)])

# Capitalize Names 
unsatisfied_prefs_df= unsatisfied_prefs_df.applymap(lambda x: x.title() if isinstance(x, str) else x)
# Remove F and A 
unsatisfied_prefs_df = unsatisfied_prefs_df.applymap(lambda x: x[:-2] if isinstance(x, str) and (x.endswith(" F") or x.endswith(" A")) else x)
# Change None to Empty
unsatisfied_prefs_df = unsatisfied_prefs_df.fillna('')
# # Group by the Role and then Sort satisfied prefs
# unsatisfied_prefs_df =  unsatisfied_prefs_df.groupby('Role'
#                                         ).apply(
    # lambda x: x.sort_values(['Unsatisfied Pref 2'], ascending=False))

In [ ]:
# unsatisfied_prefs_df.to_excel(r"C:\Users\Administrator\Downloads\SeatingArrangement_03282024_unsatisifed.xlsx")

In [1158]:
## Add the % satisfied 
unsatisfied_prefs_df['% Satisfied'] = np.nan

for index, row in unsatisfied_prefs_df.iterrows():
    if row['Role'] == 'Manager':
        fund = row['Name'].lower().replace(' ', '_') + '_F'
        if fund == 'colter_van_domelen_F': fund = 'colter_van domelen_F'
        if fund_prefs_dict[fund] != None:
            unsatisfied_prefs_df.loc[index, '% Satisfied'] = (len(satisfied_prefs[fund]) / len(fund_prefs_dict[fund]))
        else:
            unsatisfied_prefs_df.loc[index, '% Satisfied'] = 1
    elif row['Role'] == 'Allocator':
        allocator = row['Name'].lower().replace(' ', '_') + '_A'
        if allocator_prefs_dict[allocator] != None:
            unsatisfied_prefs_df.loc[index, '% Satisfied'] = (len(satisfied_prefs[allocator]) / len(allocator_prefs_dict[allocator]))
        else:
            unsatisfied_prefs_df.loc[index, '% Satisfied'] = 1


##### **Allocator No-Pref Pairs Dataframe**

In [1159]:
# Replace None values with empty lists
allocator_no_prefs_dict_filtered = {k: v if v is not None else [] for k, v in allocator_no_prefs_dict.items()}

# Ensure all values are lists; replace None with empty lists
allocator_no_prefs_dict_filtered = {k: (v if isinstance(v, list) else []) for k, v in allocator_no_prefs_dict_filtered.items()}

# Find the maximum length of the lists
max_len = max(len(lst) for lst in allocator_no_prefs_dict_filtered.values())

# Pad lists with NaN values to make them all the same length
padded_data = {key: lst + [np.nan] * (max_len - len(lst)) for key, lst in allocator_no_prefs_dict_filtered.items()}

# Create a DataFrame
allocator_no_prefs_raw_df = pd.DataFrame(padded_data).T
allocator_no_prefs_raw_df = allocator_no_prefs_raw_df.reset_index()


allocator_no_prefs_raw_df.columns = ['Allocator', 'Manager 1', 'Manager 2', 'Manager 3', 'Manager 4']


# Replace _ with space
allocator_no_prefs_raw_df = allocator_no_prefs_raw_df.applymap(        
    lambda x: x.replace('_', ' ') if isinstance(x, str) else x)


# Capitalize Names 
allocator_no_prefs_raw_df = allocator_no_prefs_raw_df.applymap(lambda x: x.title() if isinstance(x, str) else x)


# replace " F" with "(F)" in all columns # use positive lookback assertion
allocator_no_prefs_raw_df = allocator_no_prefs_raw_df.replace({'(?<= )F$': ''}, regex=True)
allocator_no_prefs_raw_df = allocator_no_prefs_raw_df.replace({'(?<= )A$': ''}, regex=True)
allocator_no_prefs_raw_df = allocator_no_prefs_raw_df.replace({'(?<= )S$': ''}, regex=True)

In [1160]:
allocator_no_prefs_dict_satisfied  = {}

for key in allocator_no_prefs_dict:
    if allocator_no_prefs_dict[key] != None:
        allocator_no_prefs_dict_satisfied[key] = []
        for value in allocator_no_prefs_dict[key]:
            if value == 'colter_van_domelen_F': value = 'colter_van domelen_F'
            if pair_counts[(value, key)] > 0 :
                # print((value, key))
                allocator_no_prefs_dict_satisfied[key].append(value)

allocator_no_prefs_dict_satisfied = {k: v for k, v in allocator_no_prefs_dict_satisfied.items() if v}

In [1161]:
if len(allocator_no_prefs_dict_satisfied)>0:
    # Replace None values with empty lists
    allocator_no_prefs_dict_satisfied = {k: v if v is not None else [] for k, v in allocator_no_prefs_dict_satisfied.items()}

    # Ensure all values are lists; replace None with empty lists
    allocator_no_prefs_dict_satisfied = {k: (v if isinstance(v, list) else []) for k, v in allocator_no_prefs_dict_satisfied.items()}

    # Find the maximum length of the lists
    max_len = max(len(lst) for lst in allocator_no_prefs_dict_satisfied.values())

    # Pad lists with NaN values to make them all the same length
    padded_data = {key: lst + [np.nan] * (max_len - len(lst)) for key, lst in allocator_no_prefs_dict_satisfied.items()}

    # Create a DataFrame
    allocator_no_prefs_satisfied_df = pd.DataFrame(padded_data).T
    allocator_no_prefs_satisfied_df = allocator_no_prefs_satisfied_df.reset_index()


    allocator_no_prefs_satisfied_df.columns = ['Allocator', 'Manager 1', 'Manager 2']


    # Replace _ with space
    allocator_no_prefs_satisfied_df = allocator_no_prefs_satisfied_df.applymap(        
        lambda x: x.replace('_', ' ') if isinstance(x, str) else x)


    # Capitalize Names 
    allocator_no_prefs_satisfied_df = allocator_no_prefs_satisfied_df.applymap(lambda x: x.title() if isinstance(x, str) else x)


    # replace " F" with "(F)" in all columns # use positive lookback assertion
    allocator_no_prefs_satisfied_df = allocator_no_prefs_satisfied_df.replace({'(?<= )F$': ''}, regex=True)
    allocator_no_prefs_satisfied_df = allocator_no_prefs_satisfied_df.replace({'(?<= )A$': ''}, regex=True)
    allocator_no_prefs_satisfied_df = allocator_no_prefs_satisfied_df.replace({'(?<= )S$': ''}, regex=True)
else:
    allocator_no_prefs_satisfied_df = pd.DataFrame(columns = ['Allocator', 'Manager 1', 'Manager 2'])

##### **Create dataframe showing which matching criteria was applied**

In [1162]:
data = {
    'Session Type': ['Lunch Session', 'Lunch Session', 'Lunch Session', 'Lunch Session', 'Lunch Session', 'Lunch Session', 
                     
                     'Manager/Allocator Only Session', 'Manager/Allocator Only Session', 
    
                     'Main Sessions', 'Main Sessions', 'Main Sessions',  'Main Sessions', 'Main Sessions', 'Main Sessions'],
    'Session_Number': [lunch_session, lunch_session, lunch_session, lunch_session, lunch_session, lunch_session, 
                     
                     MAonly_session, MAonly_session, 
                 
                     main_sessions, main_sessions, main_sessions, main_sessions, main_sessions, main_sessions],
    'Matching Criteria': ['Mutual Prefs', 'Allocator Prefs','Fund Prefs',   'Allocator No Prefs',   'Aum matches', 'Aum no matches', 
                                
                          'Allocator Institution type match', 'Fund Asset Class Matches', 

                        
                        'Mutual Prefs', 'Allocator Prefs', 'Fund Prefs', 'Allocator No Prefs', 'Aum matches', 'Aum no matches'],
    'Weight': [weight_max_aum_matches_lunch, weight_max_fund_prefs_lunch, weight_min_allocator_no_prefs, weight_max_allocator_prefs_lunch, weight_min_aum_no_matches_lunch, weight_max_mutual_prefs_lunch, 
             weight_max_institution_matches_MA,  weight_max_fund_aum_matches_MA, 
 
             weight_max_mutual_prefs, weight_max_allocator_prefs, weight_max_fund_prefs, weight_min_allocator_no_prefs, weight_max_aum_matches, weight_min_aum_no_matches]
}

model_weights_df = pd.DataFrame(data)


In [ ]:
# sys.exit()

## All Session Tables 

In [1163]:
# meeting_assignments_df_reset = meeting_assignments_df.drop(columns = ['Seat']).reset_index()
meeting_assignments_df_reset = meeting_assignments_df.reset_index()


# Use the melt function to unpivot the DataFrame
meeting_assignments_df_melted = pd.melt(meeting_assignments_df_reset, id_vars=['Table'], value_vars=['Session 1', 'Session 2', 'Session 3', 'Session 4', 'Session 5', 'Session 6', 'Session 7'], var_name='Session', value_name='Name')

# Reorder the columns
meeting_assignments_df_melted = meeting_assignments_df_melted[['Session', 'Table', 'Name']]

# Define a custom function to determine the role based on the name
def get_role(name):
    if '(F)' in name:
        return 'Manager'
    elif '(A)' in name:
        return 'Allocator'
    elif '(S)' in name:
        return 'Staff'
    else:
        return ''

# Apply the custom function to create the 'Role' column
meeting_assignments_df_melted['Role'] = meeting_assignments_df_melted['Name'].apply(get_role)

# Add back in the Company Name#
for index, row in meeting_assignments_df_melted.iterrows():
    name = row['Name']
    # if name.endswith('_S'): name = name[:-2]  # Remove the last three characters "_SC"/
    name = name.lower()
    name = name[:-5]
    name = name.replace(' ', '_').strip()



    if row['Role'] =='Manager':
        name = name + '_F'
        if name == 'colter_van_domelen_F': name = 'colter_van domelen_F'
        company = fund_data_db[fund_data_db['Contact'] ==name]['Company'].values[0]
        meeting_assignments_df_melted.at[index, 'Company'] = company
    if row['Role'] =='Allocator':
        name = name + '_A'
        company = allocator_data_db[allocator_data_db['Contact'] ==name]['Company'].values[0]
        meeting_assignments_df_melted.at[index, 'Company'] = company

    if row['Role'] == 'Staff':
        name = name + '_S'
        if name != 'testing_testing_S':
            company = df_excel_staff[df_excel_staff['Contact'] ==name ]['Org'].values[0]
            meeting_assignments_df_melted.at[index, 'Company'] = company
    

# Remove the marker - leave scheduling conflict marker
meeting_assignments_df_melted['Name'] = meeting_assignments_df_melted['Name'].str.replace(r'\(A\)|\(F\)||\(S\)', '', regex=True)

# # Add host information
# meeting_assignments_df_melted['HOST'] = np.where(meeting_assignments_df_melted['Role'].isin(['Manager', 'Allocator']), True, False)

IndexError: index 0 is out of bounds for axis 0 with size 0

### Add Institution Type and AUM

In [ ]:
for index, row in meeting_assignments_df_melted.iterrows():
    name = row['Name']
    name = name.strip().lower().replace(' ', '_')


    # name = name.replace(' ', '_').replace('_(a)', '_A').replace('_(f)', '_F').replace('_(s)', '_S')
    # print(name)

    if name != '-':

        if row['Role'] == 'Allocator':
            name = name + '_A'
            allocator_inst_type = allocator_data_db[allocator_data_db['Contact'] == name]['Description Of Organization'].values[0]
            allocator_aum = allocator_data_db[allocator_data_db['Contact'] == name]['AUM Range'].values[0]
            meeting_assignments_df_melted.at[index, 'Description Of Organization'] = allocator_inst_type
            meeting_assignments_df_melted.at[index, 'AUM Range'] = allocator_aum
        elif row['Role'] == "Manager":
            name = name + '_F'
            if name == 'colter_van_domelen_F': name = 'colter_van domelen_F'
            manager_aum = fund_data_db[fund_data_db['Contact'] == name]['AUM Range'].values[0]
            meeting_assignments_df_melted.at[index, 'AUM Range'] = manager_aum

In [1165]:
# Create dictionary to store pair counts # 
lunch_table_pair_counts = {}
for pair in pair_count_list:
    for table in lunch_table_assignments['Session 8']:
        assignments = list(lunch_table_assignments['Session 8'][table].values())
        if pair[0] in assignments and pair[1] in assignments:
            lunch_table_pair_counts[pair] = 1

In [1168]:
for pair in allocator_no_pref_pairs:
    if pair in lunch_table_pair_counts:
        print(pair)

In [ ]:
# meeting_assignments_df_melted[(meeting_assignments_df_melted['Session'] == 'Session 4') & (meeting_assignments_df_melted['Table'] == 11)]

In [ ]:
end_time = time.time()
print(f'This Took {round((end_time - start_time)/60, 2)} minutes')

# Write to Excel Sheet

In [1164]:
if write_to_excel == True:
    
    date = datetime.now().strftime("%m%d%Y%H%M")

    # Write DataFrames to Excel with different sheets
    with pd.ExcelWriter(rf"C:\Users\Administrator\Downloads\SeatingArrangement_{date}.xlsx") as writer:
        summary_df.to_excel(writer, sheet_name='SummaryStats', index=True)
        table_frameworks_df.to_excel(writer, sheet_name='TableFramework', index=True)
        ratio_counts_df.to_excel(writer, sheet_name='RatioCounts', index=True)
        FA_counts_df.to_excel(writer, sheet_name='FACounts', index=True)
        meeting_assignments_df.to_excel(writer, sheet_name='SeatingArrangement', index=True)
        # meeting_assignments_df_melted.to_excel(writer, sheet_name='AllSessionTables', index=True)
        raw_prefs_A_df.to_excel(writer, sheet_name = 'PrefsForAllocators', index = True)
        raw_prefs_F_df.to_excel(writer, sheet_name = 'PrefsForManagers', index = True)
        satisfied_prefs_df.to_excel(writer, sheet_name='SatisfiedPrefs', index=True)
        unsatisfied_prefs_df.to_excel(writer, sheet_name='UnsatisfiedPrefs', index=True)
        pair_counts_matrix.to_excel(writer, sheet_name='PairCountsMatrix', index=True)
        pair_counts_matrix_no8.to_excel(writer, sheet_name='PairCountsMatrix (no Lunch)', index=True)
        pair_counts_matrix_main.to_excel(writer, sheet_name='PairCountsMatrix (no Lunch MA)', index=True)
        allocator_no_prefs_raw_df.to_excel(writer, sheet_name='AllocatorNoPrefs(Raw)', index=True)
        allocator_no_prefs_satisfied_df.to_excel(writer, sheet_name='AllocatorNoPrefs(Satisfied)', index=True)



## Write to DB 
This code does not work since this is a project built on sample data and not affiliated with any true Data Source. However, I still wanted to write code to show how I would write this into a Database table if need be. 

In [ ]:
## OCDB Connection 

logins_username = ""

logins_password = ""

server =  ""

# Choose Database
database= ''

## Create Engine and Connection to the Database
engine = db.create_engine(
    'mssql+pyodbc://'+logins_username+':'+logins_password+'@'+server+'/'+database+'?driver=SQL Server', 
    use_setinputsizes=False) # This argument got rid of the Error

# Find all tables in the db 
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+logins_username+';PWD='+ logins_password)
tables = pd.read_sql("select * from INFORMATION_SCHEMA.TABLES", cnxn)

In [ ]:
# Set table name
table_name = ''


In [ ]:
# Asset Class Session
string_weights_asset_session = f'weight_max_asset_class_match1_asset = {weight_max_asset_class_match1_asset}, weight_max_asset_class_match2_asset = {weight_max_asset_class_match2_asset}, weight_min_allocator_no_prefs_asset = {weight_min_allocator_no_prefs_asset}'

# Manager Allocator Only Session
string_weights_MA_session= f'weight_max_institution_matches_MA = {weight_max_institution_matches_MA}, weight_max_fund_asset_matches_MA = {weight_max_fund_asset_matches_MA}'

# Dive Deeper Sessions
string_weights_dd_session= f'weight_max_fund_asset_matches_dd = {weight_max_fund_asset_matches_dd}, weight_max_mutual_prefs_dd = {weight_max_mutual_prefs_dd}, weight_max_allocator_prefs_dd = {weight_max_allocator_prefs_dd}, weight_max_fund_prefs_dd = {weight_max_fund_prefs_dd}, weight_min_allocator_no_prefs_dd = {weight_min_allocator_no_prefs_dd}'

# Main Sessions 
string_weights_main_session= f'weight_max_mutual_prefs = {weight_max_mutual_prefs}, weight_max_allocator_prefs = {weight_max_allocator_prefs}, weight_max_fund_prefs = {weight_max_fund_prefs}, weight_min_allocator_no_prefs = {weight_min_allocator_no_prefs}, weight_max_aum_matches = {weight_max_aum_matches}, weight_min_aum_no_matches  = {weight_min_aum_no_matches}'

In [ ]:
# tables = pd.read_sql("select * from INFORMATION_SCHEMA.TABLES", cnxn)
# tables[tables['TABLE_NAME'] == table_name]

# Subset to only needed fields 
write_meeting_assignments_df = meeting_assignments_df.copy()

# Add column with date time
write_meeting_assignments_df['Run_Date'] = datetime.now()

# Add column with Run time - want to look at how this changes as the number of meetings increases 
write_meeting_assignments_df['Run_Time(min)'] = round(((end_time - start_time)/60),2)

# Add column with weights for each consideration 
write_meeting_assignments_df['Parameters (Asset Class)'] = string_weights_asset_session
write_meeting_assignments_df['Parameters (Fund/Allocator Only)'] = string_weights_MA_session
write_meeting_assignments_df['Parameters (Dive Deeper)'] = string_weights_dd_session
write_meeting_assignments_df['Parameters (Main)'] = string_weights_main_session

write_meeting_assignments_df = write_meeting_assignments_df.reset_index() 

write_meeting_assignments_df = write_meeting_assignments_df.fillna('-')








# CHECKS IF TABLE IS THERE - if not, create it using above function.  

if table_name in pd.read_sql("select * from INFORMATION_SCHEMA.TABLES", cnxn)['TABLE_NAME'].tolist():
    # TRUNCATE TABLE 
    # cnxn.execute(f"TRUNCATE TABLE {table_name}")
    # print(f"The contents of table '{table_name}' have been deleted.")

    # DROP TABLE 
    # cnxn.execute(f"DROP TABLE {table_name}")
    # print(f"{table_name} has been deleted.")

    # APPEND to Table
    print('The table is in the DB!')
else:
    print(f"Table '{table_name}' does not exist, creating!")
    create_database_table(table_name,'write_meeting_assignments_df', write_meeting_assignments_df.columns, engine)## GOOD TO GO

cnxn.commit()






# POPULATE TABLE
# check after the possible creation of the table above if the table is there
tables = pd.read_sql("select * from INFORMATION_SCHEMA.TABLES", cnxn)
if table_name in tables['TABLE_NAME'].tolist():
    print('Table found in table names')

## Check if table exists and is empty - if so populate it ##
if pd.read_sql_query(f"SELECT OBJECT_ID('{table_name}') AS obj_id", cnxn)['obj_id'][0] is None:
    print('ERROR: Table does not exist - check previous code for Table Creation')
elif pd.read_sql_query(f"SELECT COUNT(*) AS count FROM {table_name}", cnxn)['count'][0] != 0:
    print('Table is not empty - Appending data')
    write_meeting_assignments_df.to_sql(table_name, con=engine, if_exists='append', index=False)
else:
    # If table exists and is empty, write the DataFrame to the new SQL table using to_sql()
    print(table_name, 'Table exists and is empty - populating :)')
    write_meeting_assignments_df.to_sql(table_name, con=engine, if_exists='append', index=False)